<a href="https://colab.research.google.com/github/orhanaydinn/OAix-3B-LLM-model/blob/main/Copy_of_OAIx_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Environment Setup

## 1.1 Check GPU Availability

In [ ]:
!nvidia-smi


Thu Oct 30 16:12:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 1.2 Install Required Libraries

In [1]:
!pip -q install "transformers==4.44.0" "datasets==2.21.0" "accelerate==0.34.2" \
"peft==0.12.0" "trl==0.9.6" "bitsandbytes==0.43.3" "sentencepiece==0.2.0" \
"deepspeed==0.14.4"



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 93.6 MB/s eta 0:00:00
 

In [2]:
!pip install torch==2.3.1 --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not

In [3]:
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.1 MB/s eta 0:00:00


In [4]:
!pip install -U "datasets==2.19.1" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.23.0+cu126 requires torch==2.8.0, but you have torch 2.3.1 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


In [5]:
!pip install -U "datasets==2.19.1" "pyarrow==15.0.2" "fsspec==2024.6.0" -q

ERROR: Cannot install datasets==2.19.1 and fsspec==2024.6.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


## 1.3 Create Project Directory Folder

In [ ]:
!mkdir -p /content/oaix-3b/data/raw

# 2. Tokenizer Preparation

## 2.1 Download Dataset

In [ ]:
!rm -rf ~/.cache/huggingface/datasets

In [ ]:
from datasets import load_dataset

wiki = load_dataset("wikitext", "wikitext-103-v1", split="train")

out_path = "/content/oaix-3b/data/raw/corpus.txt"
with open(out_path, "w", encoding="utf-8") as f:
    for line in wiki["text"]:
        if line.strip():
            f.write(line.strip() + "\n")

print(f"Corpus saved to: {out_path}")


Corpus saved to: /content/oaix-3b/data/raw/corpus.txt


## 2.2 Control Dataset

In [ ]:
!ls /content/oaix-3b/data/raw/

corpus.txt


## 2.3 Tokenizer Training

In [ ]:
import os
os.makedirs("/content/oaix-3b/tokenizer", exist_ok=True)
os.makedirs("/content/oaix-3b/data/raw", exist_ok=True)

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input="/content/oaix-3b/data/raw/corpus.txt",
    model_prefix="/content/oaix-3b/tokenizer/spm32k",
    vocab_size=32000,             # 32k kelime hazinesi
    model_type="bpe",             # byte pair encoding (modern LLM standardı)
    byte_fallback=True,           # tüm karakterleri tanıyabilsin
    character_coverage=0.9995,    # İngilizce için ideal
    input_sentence_size=2000000,  # rastgele 2 milyon cümle kullan
    shuffle_input_sentence=True   # veri karışık gelsin
)

print("Tokenizer training complete!")


Tokenizer training complete!


## 2.4 Test Tokenizer

In [ ]:
from transformers import LlamaTokenizer

tok = LlamaTokenizer(
    vocab_file="/content/oaix-3b/tokenizer/spm32k.model",
    tokenizer_file=None
)
tok.pad_token = tok.eos_token

print("Tokenizer loaded successfully from SentencePiece model!")


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


Tokenizer loaded successfully from SentencePiece model!


# 3. Model Configuration

## 3.1 Wrap the tokenizer for Transformers

In [ ]:
import json, os, shutil, pathlib

tok_dir = "/content/oaix-3b/tokenizer"
# spm32k.model zaten burada olmalı

# minimal tokenizer_config.json
tok_cfg = {
  "model_max_length": 4096,
  "padding_side": "right",
  "truncation_side": "right"
}
with open(os.path.join(tok_dir, "tokenizer_config.json"), "w") as f:
    json.dump(tok_cfg, f, indent=2)

print("tokenizer_config.json written.")


tokenizer_config.json written.


# 3.2 Define a ~3B LLaMA-style config

In [ ]:
import os
from transformers import LlamaConfig

cfg = LlamaConfig(
    vocab_size=32000,
    hidden_size=3072,          # ✅ 3072 / 24 = 128 (tam bölünüyor)
    intermediate_size=8192,    # oran ~2.67× (LLaMA oranı)
    num_hidden_layers=32,
    num_attention_heads=24,    # 24 Query head
    num_key_value_heads=8,     # ✅ GQA aktif (24 % 8 == 0)
    rms_norm_eps=1e-5,
    hidden_act="silu",
    max_position_embeddings=4096,
    rope_theta=10000.0,
    initializer_range=0.02,
    use_cache=True,
    tie_word_embeddings=False
)

save_dir = "/content/oaix-3b/ckpts/config_3b"
os.makedirs(save_dir, exist_ok=True)
cfg.save_pretrained(save_dir)
print("✅ Updated 3B config (GQA ON, mathematically aligned) saved:", save_dir)


✅ Updated 3B config (GQA ON, mathematically aligned) saved: /content/oaix-3b/ckpts/config_3b


## 3.3 Parameter Estimation (Model Size Check)

In [ ]:
%reset -f

In [ ]:
def estimate_llama_params(vocab_size, d, L, d_ff):
    embed = vocab_size * d
    transformer = L * (2 * (d ** 2) + 4 * d_ff * d)  # daha doğru katsayı
    total = embed + transformer
    return total

vocab_size = 32000
d = 2850
L = 30
d_ff = 7200

approx = estimate_llama_params(vocab_size, d, L, d_ff)
print(f"≈ {approx/1e9:.2f} B parameters (approx)")

≈ 3.04 B parameters (approx)


# 4. Lite Pretraining Setup

## 4.1 Train / Validation Split

In [ ]:
# ====================================================
# 4.1 — Train / Validation Split (Final Fixed)
# ====================================================

# Hugging Face datasets modülünü Python 3.12 uyumlu sürüme sabitle
!pip install -U "datasets==2.19.1" -q

from datasets import load_dataset

# --- 1️⃣ Veri yolunu ayarla ---
data_path = "/content/oaix-3b/data/raw/corpus.txt"

# --- 2️⃣ Dataset'i yükle ---
# "all" anahtarını kullanmıyoruz çünkü rezerve edilmiş bir isim.
dataset = load_dataset("text", data_files={"train": data_path})

# --- 3️⃣ %90 train, %10 validation olarak ayır ---
split = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
val_ds   = split["test"]

print(f"✅ Dataset loaded and split successfully!")
print(f"   • Train examples: {len(train_ds):,}")
print(f"   • Validation examples: {len(val_ds):,}")

# --- 4️⃣ Boş veya sadece boşluk karakteri içeren satırları filtrele ---
def not_empty(example):
    return example["text"].strip() != ""

train_ds = train_ds.filter(not_empty)
val_ds   = val_ds.filter(not_empty)

print(f"🧹 After cleaning:")
print(f"   • Train = {len(train_ds):,}")
print(f"   • Val   = {len(val_ds):,}")

# --- 5️⃣ Küçük bir örnek göster ---
print("\n📝 Example text sample:")
print(train_ds[0]["text"][:200])


✅ Dataset loaded and split successfully!
   • Train examples: 1,048,526
   • Validation examples: 116,503
🧹 After cleaning:
   • Train = 1,048,526
   • Val   = 116,503

📝 Example text sample:
Red Hot Chili Peppers


## 4.2 Tokenization & Encoding

In [ ]:
!rm -rf /content/oaix-3b/data/encoded/len512_fast_tok32000


In [ ]:
# ====================================================
# 4.2 — Tokenization & Encoding (Fixed Alignment v3 + Multi-Core FAST)
# ====================================================

import os, json, shutil, tempfile, multiprocessing
from transformers import LlamaTokenizer

# --- 1️⃣ Tokenizer Dizini ---
base_tok_dir = "/content/oaix-3b/tokenizer"
hf_tok_dir   = "/content/oaix-3b/tokenizer_hf"
os.makedirs(base_tok_dir, exist_ok=True)
os.makedirs(hf_tok_dir, exist_ok=True)

# --- 2️⃣ Tokenizer'ı Yükle ve HF Uyumu Sağla ---
tok = LlamaTokenizer(vocab_file=f"{base_tok_dir}/spm32k.model", use_fast=True)

tok.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "</s>",
})
tok.pad_token = tok.eos_token

# HF yapılandırmaları kaydet
tok_cfg = {
    "model_type": "llama",
    "tokenizer_class": "LlamaTokenizer",
    "unk_token": "<unk>",
    "bos_token": "<s>",
    "eos_token": "</s>",
    "pad_token": "</s>",
    "model_max_length": 4096,
    "padding_side": "right",
    "truncation_side": "right",
}
with open(os.path.join(base_tok_dir, "config.json"), "w") as f:
    json.dump(tok_cfg, f, indent=2)
with open(os.path.join(base_tok_dir, "tokenizer_config.json"), "w") as f:
    json.dump({
        "model_max_length": 4096,
        "padding_side": "right",
        "truncation_side": "right",
        "special_tokens_map_file": None
    }, f, indent=2)

tok.save_pretrained(base_tok_dir)
tok.save_pretrained(hf_tok_dir)
shutil.copy(f"{base_tok_dir}/spm32k.model", f"{hf_tok_dir}/spm32k.model")

print("✅ Tokenizer ready:")
print(f" • Base: {base_tok_dir}")
print(f" • HF copy: {hf_tok_dir}")
print(f" • Vocab size: {len(tok):,}")

# --- 3️⃣ Encoding Ayarları ---
MAX_LEN = 512
enc_root = "/content/oaix-3b/data/encoded"
enc_dir = f"{enc_root}/len{MAX_LEN}_fast_tok{len(tok)}_fixed"
os.makedirs(enc_root, exist_ok=True)

# Yeni cache klasörü (önceki hash çakışmasını önler)
os.environ["HF_DATASETS_CACHE"] = tempfile.mkdtemp(prefix="hf_ds_cache_")

# --- 4️⃣ Alignment-Safe Tokenization Fonksiyonu ---
def encode_example(batch):
    enc = tok(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN - 2,  # BOS + EOS eklenecek
        padding=False,
        add_special_tokens=False,
    )

    input_ids_batch, attn_mask_batch = [], []
    for ids in enc["input_ids"]:
        # BOS/EOS ekle
        ids = [tok.bos_token_id] + ids + [tok.eos_token_id]
        # Uzunluk kontrolü
        ids = ids[:MAX_LEN]
        attn = [1] * len(ids)
        # Pad gerekiyorsa uygula
        pad_len = MAX_LEN - len(ids)
        if pad_len > 0:
            ids += [tok.pad_token_id] * pad_len
            attn += [0] * pad_len
        # Güvenlik kontrolü
        assert len(ids) == MAX_LEN and len(attn) == MAX_LEN, "❌ Alignment mismatch!"
        input_ids_batch.append(ids)
        attn_mask_batch.append(attn)

    return {
        "input_ids": input_ids_batch,
        "attention_mask": attn_mask_batch,
        "labels": input_ids_batch,
    }

# --- 5️⃣ Çok Çekirdekli Encoding ---
num_proc = max(2, multiprocessing.cpu_count() // 2)
print(f"🧠 Using {num_proc} CPU workers for tokenization...")

print("🔄 Encoding train dataset...")
train_enc = train_ds.map(
    encode_example,
    batched=True,
    batch_size=64,
    num_proc=num_proc,
    remove_columns=["text"],
    load_from_cache_file=False,
    desc="Encoding train set",
)

print("🔄 Encoding validation dataset...")
val_enc = val_ds.map(
    encode_example,
    batched=True,
    batch_size=64,
    num_proc=num_proc,
    remove_columns=["text"],
    load_from_cache_file=False,
    desc="Encoding validation set",
)

# --- 6️⃣ Encode Edilmiş Dataset'i Kaydet ---
os.makedirs(enc_dir, exist_ok=True)
train_enc.save_to_disk(f"{enc_dir}/train")
val_enc.save_to_disk(f"{enc_dir}/val")

print("\n✅ Encoded datasets saved to:", enc_dir)
print(f"Train size: {len(train_enc):,} | Val size: {len(val_enc):,}")

# --- 7️⃣ Hızlı Kontrol ---
example = train_enc[0]
print("\n🧩 Sample Encoded Example:")
print("input_ids[:10] =", example["input_ids"][:10])
print("attention_mask[:10] =", example["attention_mask"][:10])
print("✅ Alignment check:", len(example["input_ids"]) == len(example["attention_mask"]) == MAX_LEN)


✅ Tokenizer ready:
 • Base: /content/oaix-3b/tokenizer
 • HF copy: /content/oaix-3b/tokenizer_hf
 • Vocab size: 32,000
🧠 Using 6 CPU workers for tokenization...
🔄 Encoding train dataset...


Encoding train set (num_proc=6):   0%|          | 0/1048526 [00:00<?, ? examples/s]

🔄 Encoding validation dataset...


Encoding validation set (num_proc=6):   0%|          | 0/116503 [00:00<?, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/1048526 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/116503 [00:00<?, ? examples/s]


✅ Encoded datasets saved to: /content/oaix-3b/data/encoded/len512_fast_tok32000_fixed
Train size: 1,048,526 | Val size: 116,503

🧩 Sample Encoded Example:
input_ids[:10] = [1, 2410, 3006, 23040, 27578, 2, 2, 2, 2, 2]
attention_mask[:10] = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
✅ Alignment check: True


### 4.2.1 Verify Encoded Datasets

In [ ]:
# ====================================================
# 4.2.1 — Verify Encoded Datasets (Sanity Check)
# ====================================================

from datasets import load_from_disk
import os

enc_dir = "/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed"  # veya yeni oluşturduğun dizin

# 1️⃣ Train ve Val dizinleri mevcut mu?
print("📂 Checking encoded dataset directories...")
print(os.listdir(enc_dir))

# 2️⃣ Hugging Face load_from_disk ile yükleme testi
try:
    train_enc = load_from_disk(os.path.join(enc_dir, "train"))
    val_enc   = load_from_disk(os.path.join(enc_dir, "val"))
    print(f"✅ Encoded datasets loaded successfully!")
    print(f"   • Train samples: {len(train_enc):,}")
    print(f"   • Val samples:   {len(val_enc):,}")
except Exception as e:
    print("❌ Error loading encoded datasets:")
    print(e)

# 3️⃣ Örnek kontrol
if 'train_enc' in locals():
    sample = train_enc[0]
    print("\n🧩 Sample check:")
    print(f"input_ids[:10] = {sample['input_ids'][:10]}")
    print(f"attention_mask[:10] = {sample['attention_mask'][:10]}")


📂 Checking encoded dataset directories...
['val', 'train']
✅ Encoded datasets loaded successfully!
   • Train samples: 1,048,526
   • Val samples:   116,503

🧩 Sample check:
input_ids[:10] = [1, 2410, 3006, 23040, 27578, 2, 2, 2, 2, 2]
attention_mask[:10] = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


# 4.3 – Deepspeed & Trainer Config

### 4.3.0 - Config + Tokenizer Sencronization

In [ ]:
# ====================================================
# 4.3.0 — Model Config + Tokenizer Synchronization
# ====================================================

from transformers import LlamaConfig, LlamaTokenizer

# Model config yolunu yükle
cfg = LlamaConfig.from_pretrained("/content/oaix-3b/ckpts/config_3b")

# Tokenizer ile senkronizasyon (zorunlu adım)
hf_tok_dir = "/content/oaix-3b/tokenizer_hf"
tok = LlamaTokenizer.from_pretrained(hf_tok_dir, use_fast=True)
tok.pad_token = tok.eos_token  # LLaMA tarzı pad = </s>

# Config’i tokenizer ile hizala
cfg.vocab_size   = len(tok)
cfg.pad_token_id = tok.pad_token_id
cfg.bos_token_id = tok.bos_token_id
cfg.eos_token_id = tok.eos_token_id

# Güvenlik: pozisyon embedding en az 512 olmalı
if getattr(cfg, "max_position_embeddings", None) and cfg.max_position_embeddings < 512:
    cfg.max_position_embeddings = 512

print(f"✅ Config synchronized with tokenizer — vocab_size={cfg.vocab_size:,}")
print(f"pad={cfg.pad_token_id}, bos={cfg.bos_token_id}, eos={cfg.eos_token_id}")


✅ Config synchronized with tokenizer — vocab_size=32,000
pad=2, bos=1, eos=2


### 4.3.1 Deepspeed config

In [ ]:
# ====================================================
# 4.3.1 — DeepSpeed Config (A100 Lite Setup)
# ====================================================

import os, json, gc, torch

# Eski cache ve DeepSpeed build klasörlerini temizle
!rm -rf /root/.cache/torch_extensions
!rm -rf /content/oaix-3b/deepspeed

gc.collect()
torch.cuda.empty_cache()

# A100 için optimize edilmiş ZeRO-2 + bf16 DeepSpeed yapılandırması
ds_config = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {"enabled": True},
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {"device": "none"},
        "offload_param": {"device": "none"}
    },
    "gradient_clipping": 1.0,
    "steps_per_print": 200,
    "wall_clock_breakdown": False
}

cfg_path = "/content/oaix-3b/configs/deepspeed_a100_lite.json"
os.makedirs(os.path.dirname(cfg_path), exist_ok=True)

with open(cfg_path, "w") as f:
    json.dump(ds_config, f, indent=2)

print(f"✅ DeepSpeed lite config saved to: {cfg_path}")


✅ DeepSpeed lite config saved to: /content/oaix-3b/configs/deepspeed_a100_lite.json


### 4.3.2 Training Config

In [ ]:
# ====================================================
# 4.3.2 — Training Config (Model + Trainer Setup, GQA ON)
# ====================================================

from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
import torch, os

# --- 1️⃣ Dataset yolları ---
enc_dir = "/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed"
ds_cfg  = "/content/oaix-3b/configs/deepspeed_a100_lite.json"

# ====================================================
# 🔧 2️⃣ HEADS kontrolü (GQA aktif)
# ====================================================

print("Before model init:")
print("  num_attention_heads:", getattr(cfg, "num_attention_heads", None))
print("  num_key_value_heads:", getattr(cfg, "num_key_value_heads", None))

# Head alignment doğrulama
assert cfg.hidden_size % cfg.num_attention_heads == 0, \
    f"❌ hidden_size ({cfg.hidden_size}) must be divisible by num_attention_heads ({cfg.num_attention_heads})"
assert cfg.num_attention_heads % cfg.num_key_value_heads == 0, \
    f"❌ num_attention_heads ({cfg.num_attention_heads}) must be divisible by num_key_value_heads ({cfg.num_key_value_heads})"

print("✅ Head configuration OK — GQA active.")
print(f"   → Q heads = {cfg.num_attention_heads}, KV heads = {cfg.num_key_value_heads}")

# ====================================================
# 3️⃣ Modeli config'ten oluştur
# ====================================================

model = AutoModelForCausalLM.from_config(cfg)

# Embedding boyutlarını tokenizer ile eşitle
model.resize_token_embeddings(len(tok))

# Gradient checkpointing aktif, cache kapalı
model.config.use_cache = False
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

print(f"✅ Model initialized — vocab_size={cfg.vocab_size:,}")

# ====================================================
# 4️⃣ Dataset'leri yükle
# ====================================================
train_enc = load_from_disk(os.path.join(enc_dir, "train"))
val_enc   = load_from_disk(os.path.join(enc_dir, "val"))

print(f"📦 Encoded datasets loaded successfully:")
print(f"   • Train: {len(train_enc):,} samples")
print(f"   • Val:   {len(val_enc):,} samples")

# ====================================================
# 5️⃣ Data Collator
# ====================================================
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

# ====================================================
# 6️⃣ TrainingArguments
# ====================================================
args = TrainingArguments(
    output_dir="/content/oaix-3b/ckpts/pretrain_lite_a100",
    per_device_train_batch_size=2,        # A100 için güvenli
    gradient_accumulation_steps=2,        # efektif batch = 4
    learning_rate=3e-5,
    warmup_steps=50,
    num_train_epochs=1,
    logging_steps=20,
    save_strategy="no",
    eval_strategy="no",
    bf16=True,                            # A100'de bf16 destekli
    gradient_checkpointing=True,
    deepspeed=ds_cfg,
    optim="adamw_torch",
    report_to="none",
    remove_unused_columns=False,
    max_grad_norm=1.0,
    dataloader_pin_memory=True,
)

# ====================================================
# 7️⃣ Trainer oluştur
# ====================================================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_enc,
    eval_dataset=val_enc,
    tokenizer=tok,
    data_collator=collator,
)

# ====================================================
# 8️⃣ Hızlı sağlık kontrolü
# ====================================================
vocab = len(tok)
emb = model.get_input_embeddings().weight.shape[0]
lm  = model.get_output_embeddings().weight.shape[0]

print(f"🔍 Vocab/Emb/LM sizes: {vocab}/{emb}/{lm}")
assert vocab == emb == lm, "❌ Vocab-embedding-lm_head uyumsuz! (tensor mismatch nedeni)"

# Batch test (early check)
sample = collator([train_enc[0], train_enc[1]])
print("✅ Trainer ready — batch sample shapes:")
print({k: v.shape for k, v in sample.items()})


Before model init:
  num_attention_heads: 24
  num_key_value_heads: 8
✅ Head configuration OK — GQA active.
   → Q heads = 24, KV heads = 8
✅ Model initialized — vocab_size=32,000
📦 Encoded datasets loaded successfully:
   • Train: 1,048,526 samples
   • Val:   116,503 samples
🔍 Vocab/Emb/LM sizes: 32000/32000/32000
✅ Trainer ready — batch sample shapes:
{'input_ids': torch.Size([2, 512]), 'attention_mask': torch.Size([2, 512]), 'labels': torch.Size([2, 512])}


## Save Google Drive

In [ ]:
# ====================================================
# BACKUP — Save All Project Files to Google Drive
# ====================================================

from google.colab import drive
import shutil, os

# --- 1️ Connect Google Drive ---
drive.mount('/content/drive')

# --- 2️ Backup Folder Path ---
backup_dir = "/content/drive/MyDrive/OAIx_3B_Backup"
os.makedirs(backup_dir, exist_ok=True)

print(f"Google Drive mounted. Backup folder: {backup_dir}")

# --- 3️ Backup Folder Name ---
folders_to_backup = [
    "/content/oaix-3b/tokenizer",
    "/content/oaix-3b/tokenizer_hf",
    "/content/oaix-3b/data/encoded/len512_fast_tok32000",
    "/content/oaix-3b/configs",
    "/content/oaix-3b/ckpts"
]


for folder in folders_to_backup:
    if os.path.exists(folder):
        target = os.path.join(backup_dir, os.path.basename(folder))
        if os.path.exists(target):
            print(f"Updating existing backup: {target}")
            shutil.rmtree(target)
        shutil.copytree(folder, target)
        print(f"Backed up: {folder} → {target}")
    else:
        print(f"Folder not found, skipping: {folder}")

print("\nBackup complete! All important files are safe in Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted. Backup folder: /content/drive/MyDrive/OAIx_3B_Backup
Updating existing backup: /content/drive/MyDrive/OAIx_3B_Backup/tokenizer
Backed up: /content/oaix-3b/tokenizer → /content/drive/MyDrive/OAIx_3B_Backup/tokenizer
Updating existing backup: /content/drive/MyDrive/OAIx_3B_Backup/tokenizer_hf
Backed up: /content/oaix-3b/tokenizer_hf → /content/drive/MyDrive/OAIx_3B_Backup/tokenizer_hf
Folder not found, skipping: /content/oaix-3b/data/encoded/len512_fast_tok32000
Updating existing backup: /content/drive/MyDrive/OAIx_3B_Backup/configs
Backed up: /content/oaix-3b/configs → /content/drive/MyDrive/OAIx_3B_Backup/configs
Updating existing backup: /content/drive/MyDrive/OAIx_3B_Backup/ckpts
Backed up: /content/oaix-3b/ckpts → /content/drive/MyDrive/OAIx_3B_Backup/ckpts

Backup complete! All important files are safe in Google Drive.


## 4.4 — Lite Pretraining Test

In [ ]:
# ====================================================
# 🧩 TruncatingCollator (runtime'da sequence kısaltma)
# ====================================================

from transformers import DataCollatorForLanguageModeling

class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=256, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length

    def __call__(self, examples):
        batch = super().__call__(examples)
        # Tüm tensörleri max_length'e kırp
        for k in batch:
            if batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch

# Collator'ı oluştur ve Trainer'a ata
collator = TruncatingCollator(tokenizer=tok, max_length=256)
trainer.data_collator = collator

# 16-bit precision ile VRAM'i düşür
trainer.args.fp16 = True
trainer.args.bf16 = False

print("✅ TruncatingCollator aktif — runtime'da 256 token ile çalışacak.")


✅ TruncatingCollator aktif — runtime'da 256 token ile çalışacak.


In [ ]:
# ====================================================
# 🚀 TRAIN — Lite Pretraining (Drive Save + OOM-SAFE + Loss Monitor)
# ====================================================

from google.colab import drive
import torch, os, time

# ✅ Google Drive bağla
drive.mount("/content/drive", force_remount=True)
drive_ckpt_dir = "/content/drive/MyDrive/OAIx_3B_Checkpoints/lite_run"
os.makedirs(drive_ckpt_dir, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {drive_ckpt_dir}")

# ====================================================
# 🧩 Runtime'da sequence truncate eden collator
# ====================================================
from transformers import DataCollatorForLanguageModeling

class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=256, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length

    def __call__(self, examples):
        batch = super().__call__(examples)
        for k in batch:
            if batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch

collator = TruncatingCollator(tokenizer=tok, max_length=256)
trainer.data_collator = collator

# ====================================================
# 🔧 Trainer argümanlarını düzenle
# ====================================================
trainer.args.fp16 = True
trainer.args.bf16 = False
trainer.args.output_dir = drive_ckpt_dir
trainer.args.save_strategy = "steps"
trainer.args.save_steps = 500
trainer.args.logging_steps = 10
trainer.args.save_total_limit = 3
trainer.args.eval_strategy = "no"
trainer.args.logging_first_step = True

print("\n✅ Trainer configured for Drive checkpointing:")
print(f" • Output dir: {trainer.args.output_dir}")
print(f" • Save every {trainer.args.save_steps} steps (DeepSpeed sharded)")
print(f" • Log every {trainer.args.logging_steps} steps")
print(f" • Save total limit: {trainer.args.save_total_limit}")
print(f" • Precision: fp16={trainer.args.fp16}, bf16={trainer.args.bf16}")
print(f" • Runtime truncate length: {collator.max_length}")

# ====================================================
# 🧩 Callback: her log step'te loss çıktısı
# ====================================================
from transformers import TrainerCallback

class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            step = state.global_step
            loss = logs["loss"]
            print(f"Step {step:<6} | Loss: {loss:.4f}")

trainer.add_callback(LossPrinterCallback())

# ====================================================
# 💾 Eğitimi başlat — DeepSpeed checkpoint + Drive save + Loss log
# ====================================================
try:
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
    train_output = trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("❌ OOM detected — reduce sequence length or enable CPU offload.")
        torch.cuda.empty_cache()
        train_output = None
    else:
        raise e

# ====================================================
# 💾 Eğitim sonunda final checkpoint (Drive'a)
# ====================================================
if train_output is not None:
    final_dir = os.path.join(drive_ckpt_dir, "final_sharded")
    os.makedirs(final_dir, exist_ok=True)
    if hasattr(trainer, "deepspeed") and trainer.deepspeed is not None:
        print("💾 Saving final sharded checkpoint via DeepSpeed…")
        trainer.deepspeed.save_checkpoint(final_dir, tag="final")
    else:
        print("⚠️ No DeepSpeed engine; skipping final sharded save.")
    try:
        trainer.save_state()
    except Exception as e:
        print(f"⚠️ save_state skipped: {e}")
    print(f"\n✅ Lite pretraining finished. Sharded checkpoints saved to Drive: {final_dir}")
else:
    print("\n❌ Training failed.")

# ====================================================
#  📊 Sonuç Özeti
# ====================================================
if train_output is not None:
    metrics = train_output.metrics
    print("\n📊 Training summary:")
    print(f" • Runtime: {metrics.get('train_runtime', 'N/A')}")
    print(f" • Final Loss: {metrics.get('train_loss', 'N/A')}")
    print(f" • Samples/sec: {metrics.get('train_samples_per_second', 'N/A')}")


Mounted at /content/drive
✅ Checkpoints will be saved to: /content/drive/MyDrive/OAIx_3B_Checkpoints/lite_run

✅ Trainer configured for Drive checkpointing:
 • Output dir: /content/drive/MyDrive/OAIx_3B_Checkpoints/lite_run
 • Save every 500 steps (DeepSpeed sharded)
 • Log every 10 steps
 • Save total limit: 3
 • Precision: fp16=True, bf16=False
 • Runtime truncate length: 256


Step,Training Loss
1,7.722700
10,6.042900
20,5.888500
30,6.592100
40,6.295300
50,5.417000
60,6.430700
70,5.554400
80,5.957000
90,5.545700


Step 1      | Loss: 7.7227
Step 10     | Loss: 6.0429
Step 20     | Loss: 5.8885
Step 30     | Loss: 6.5921
Step 40     | Loss: 6.2953
Step 50     | Loss: 5.4170
Step 60     | Loss: 6.4307
Step 70     | Loss: 5.5544
Step 80     | Loss: 5.9570
Step 90     | Loss: 5.5457
Step 100    | Loss: 5.8371
Step 110    | Loss: 5.7392
Step 120    | Loss: 6.1420
Step 130    | Loss: 6.7366
Step 140    | Loss: 6.0353
Step 150    | Loss: 6.8877
Step 160    | Loss: 6.8174
Step 170    | Loss: 6.6231
Step 180    | Loss: 5.3898
Step 190    | Loss: 5.9201
Step 200    | Loss: 6.3132
Step 210    | Loss: 5.2750
Step 220    | Loss: 5.4109
Step 230    | Loss: 5.2254
Step 240    | Loss: 6.3216
Step 250    | Loss: 5.5044
Step 260    | Loss: 5.6397
Step 270    | Loss: 5.5913
Step 280    | Loss: 6.2844
Step 290    | Loss: 5.3928
Step 300    | Loss: 6.1170
Step 310    | Loss: 4.9452
Step 320    | Loss: 4.6317
Step 330    | Loss: 5.0261
Step 340    | Loss: 5.8727
Step 350    | Loss: 5.1208
Step 360    | Loss: 6.2090
S

# 5.0 Full Pretraining Phase 1 (0-25)

## 5.1 Load All Files from Drive

In [ ]:
# ====================================================
# RESTORE — Load Project Files Back from Google Drive
# ====================================================

from google.colab import drive
import os, shutil

# --- 1️⃣ Drive bağlantısı ---
drive.mount('/content/drive')

# --- 2️⃣ Drive üzerindeki yedek klasör ---
backup_dir = "/content/drive/MyDrive/OAIx_3B_Backup"
local_dir = "/content/oaix-3b"

# --- 3️⃣ /content dizininde yeni proje klasörü oluştur ---
if os.path.exists(local_dir):
    shutil.rmtree(local_dir)
os.makedirs(local_dir, exist_ok=True)

# --- 4️⃣ Kopyalanacak klasörler ---
folders_to_restore = [
    "tokenizer",
    "tokenizer_hf",
    "configs",
    "ckpts",
    "data"
]

# --- 5️⃣ Geri yükleme işlemi ---
for folder in folders_to_restore:
    src = os.path.join(backup_dir, folder)
    dst = os.path.join(local_dir, folder)
    if os.path.exists(src):
        shutil.copytree(src, dst)
        print(f"✅ Restored: {src} → {dst}")
    else:
        print(f"⚠️ Folder not found in Drive (skipped): {src}")

print("\n📦 All project files restored successfully!")
!ls -lh /content/oaix-3b


Mounted at /content/drive
✅ Restored: /content/drive/MyDrive/OAIx_3B_Backup/tokenizer → /content/oaix-3b/tokenizer
✅ Restored: /content/drive/MyDrive/OAIx_3B_Backup/tokenizer_hf → /content/oaix-3b/tokenizer_hf
✅ Restored: /content/drive/MyDrive/OAIx_3B_Backup/configs → /content/oaix-3b/configs
✅ Restored: /content/drive/MyDrive/OAIx_3B_Backup/ckpts → /content/oaix-3b/ckpts
⚠️ Folder not found in Drive (skipped): /content/drive/MyDrive/OAIx_3B_Backup/data

📦 All project files restored successfully!
total 16K
drwx------ 5 root root 4.0K Oct 30 03:00 ckpts
drwx------ 2 root root 4.0K Oct 30 02:22 configs
drwx------ 2 root root 4.0K Oct 30 02:10 tokenizer
drwx------ 2 root root 4.0K Oct 30 02:10 tokenizer_hf


In [ ]:
import os, shutil

drive_encoded = "/content/drive/MyDrive/OAIx_3B_Backup/len512_fast_tok32000"
local_encoded = "/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed"

# Hedef dizin oluştur
os.makedirs("/content/oaix-3b/data", exist_ok=True)

# Mevcut encoded dizin varsa sil (temiz başlatmak için)
if os.path.exists(local_encoded):
    shutil.rmtree(local_encoded)

# Drive'dan kopyala
shutil.copytree(drive_encoded, local_encoded)
print(f"✅ Restored encoded dataset to: {local_encoded}")

# Kontrol et
!ls -lh {local_encoded}


✅ Restored encoded dataset to: /content/oaix-3b/data/encoded/len512_fast_tok32000_fixed
total 8.0K
drwx------ 2 root root 4.0K Oct 30 03:03 train
drwx------ 2 root root 4.0K Oct 30 02:59 val


In [ ]:
# ====================================================
# Reload Model Config + Tokenizer
# ====================================================
from transformers import LlamaConfig, LlamaTokenizer, AutoModelForCausalLM

cfg_path = "/content/oaix-3b/ckpts/config_3b"
hf_tok_dir = "/content/oaix-3b/tokenizer_hf"

# Config ve tokenizer'ı yeniden yükle
cfg = LlamaConfig.from_pretrained(cfg_path)
tok = LlamaTokenizer.from_pretrained(hf_tok_dir, use_fast=True)
tok.pad_token = tok.eos_token

# Model oluştur ve hizala
model = AutoModelForCausalLM.from_config(cfg)
model.resize_token_embeddings(len(tok))
model.config.use_cache = False
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

print(f"✅ Model + Tokenizer reloaded — vocab_size={len(tok):,}")


✅ Model + Tokenizer reloaded — vocab_size=32,000


In [ ]:
# ====================================================
# OAIx-3B — Phase 1 Real Training (25% Dataset + Auto Resume)
# ====================================================

from google.colab import drive
import torch, os, time, re
from transformers import DataCollatorForLanguageModeling, TrainerCallback
from datasets import DatasetDict, load_from_disk

# ====================================================
# 1️⃣ Google Drive bağla
# ====================================================
drive.mount("/content/drive", force_remount=True)
drive_ckpt_dir = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints"
os.makedirs(drive_ckpt_dir, exist_ok=True)
print(f"Checkpoints will be saved to: {drive_ckpt_dir}")

# ====================================================
# 2️⃣ Dataset yükle ve %25 subset oluştur
# ====================================================
encoded_dataset = DatasetDict({
    "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
    "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
})

train_data = encoded_dataset["train"]
val_data   = encoded_dataset["val"]

train_len = len(train_data)
val_len   = len(val_data)

subset_train = train_data.select(range(0, int(0.25 * train_len)))
subset_val   = val_data.select(range(0, int(0.25 * val_len)))

print(f"Subset hazır: {len(subset_train):,}/{train_len:,} train | {len(subset_val):,}/{val_len:,} val")

trainer.train_dataset = subset_train
trainer.eval_dataset  = subset_val

# ====================================================
# 3️⃣ Runtime'da truncate eden collator (256 token)
# ====================================================
class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=256, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length
    def __call__(self, examples):
        batch = super().__call__(examples)
        for k in batch:
            if batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch

collator = TruncatingCollator(tokenizer=tok, max_length=256)
trainer.data_collator = collator

# ====================================================
# 4️⃣ TrainingArguments — gerçek koşu
# ====================================================
trainer.args.output_dir = drive_ckpt_dir
trainer.args.save_strategy = "steps"
trainer.args.save_steps = 2000       # 🔁 her 2000 adımda checkpoint
trainer.args.save_total_limit = 3
trainer.args.logging_steps = 50
trainer.args.eval_strategy = "no"
trainer.args.fp16 = True
trainer.args.bf16 = False
trainer.args.report_to = "none"
trainer.args.max_steps = 10000       # 🔧 toplam adım sayını buradan belirle

print("\nTrainer configured for REAL training:")
print(f" • Dataset: First 25% subset")
print(f" • Output dir: {trainer.args.output_dir}")
print(f" • Save every {trainer.args.save_steps} steps (DeepSpeed sharded)")
print(f" • Save total limit: {trainer.args.save_total_limit}")
print(f" • Max steps: {trainer.args.max_steps}")
print(f" • Runtime truncate length: {collator.max_length}")

# ====================================================
# 5️⃣ Son checkpoint’i Drive'dan otomatik bul (resume desteği)
# ====================================================
def find_latest_ckpt(base_dir):
    ckpts = [d for d in os.listdir(base_dir) if d.startswith("checkpoint-")]
    if not ckpts:
        return None
    steps = sorted([int(re.findall(r"\d+", c)[0]) for c in ckpts])
    latest = os.path.join(base_dir, f"checkpoint-{steps[-1]}")
    return latest

latest_ckpt = find_latest_ckpt(drive_ckpt_dir)
if latest_ckpt:
    print(f"Found checkpoint: {latest_ckpt}")
else:
    print("No previous checkpoint found — starting from scratch.")

# ====================================================
# 6️⃣ Loss log callback
# ====================================================
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")

trainer.add_callback(LossPrinterCallback())

# ====================================================
# 7️⃣ Training başlat (resume destekli)
# ====================================================
try:
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
    if latest_ckpt:
        print(f"Resuming OAIx-3B Phase-1 Training from: {latest_ckpt}")
        train_output = trainer.train(resume_from_checkpoint=latest_ckpt)
    else:
        print("Starting OAIx-3B Phase-1 Training (25% dataset)...")
        train_output = trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("OOM detected — try lower seq_len or CPU offload.")
        torch.cuda.empty_cache()
        train_output = None
    else:
        raise e

# ====================================================
# 8️⃣ Final checkpoint (Drive’a kaydet)
# ====================================================
if train_output is not None:
    final_dir = os.path.join(drive_ckpt_dir, "final_state1_sharded")
    os.makedirs(final_dir, exist_ok=True)
    if hasattr(trainer, "deepspeed") and trainer.deepspeed is not None:
        trainer.deepspeed.save_checkpoint(final_dir, tag="final_state1")
        print(f"Final sharded checkpoint saved to: {final_dir}")
    try:
        trainer.save_state()
    except Exception as e:
        print(f"save_state skipped: {e}")
    print("\nOAIx-3B Phase-1 training finished successfully.")
else:
    print("\nTraining failed.")

# ====================================================
# 9️⃣ Özet
# ====================================================
if train_output is not None:
    metrics = train_output.metrics
    print("\n Training summary:")
    print(f" • Runtime: {metrics.get('train_runtime', 'N/A')}")
    print(f" • Final Loss: {metrics.get('train_loss', 'N/A')}")
    print(f" • Samples/sec: {metrics.get('train_samples_per_second', 'N/A')}")


Mounted at /content/drive
Checkpoints will be saved to: /content/drive/MyDrive/OAIx_3B_state1_checkpoints
Subset hazır: 262,131/1,048,526 train | 29,125/116,503 val

Trainer configured for REAL training:
 • Dataset: First 25% subset
 • Output dir: /content/drive/MyDrive/OAIx_3B_state1_checkpoints
 • Save every 2000 steps (DeepSpeed sharded)
 • Save total limit: 3
 • Max steps: 10000
 • Runtime truncate length: 256
No previous checkpoint found — starting from scratch.
Starting OAIx-3B Phase-1 Training (25% dataset)...


Step,Training Loss
1,8.815600
50,6.822300
100,6.626900
150,6.743700
200,6.438200
250,6.786600
300,6.220200
350,6.752600
400,6.887300
450,7.075500


Step 1      | Loss: 8.8156
Step 1      | Loss: 8.8156
Step 50     | Loss: 6.8223
Step 50     | Loss: 6.8223
Step 100    | Loss: 6.6269
Step 100    | Loss: 6.6269
Step 150    | Loss: 6.7437
Step 150    | Loss: 6.7437
Step 200    | Loss: 6.4382
Step 200    | Loss: 6.4382
Step 250    | Loss: 6.7866
Step 250    | Loss: 6.7866
Step 300    | Loss: 6.2202
Step 300    | Loss: 6.2202
Step 350    | Loss: 6.7526
Step 350    | Loss: 6.7526
Step 400    | Loss: 6.8873
Step 400    | Loss: 6.8873
Step 450    | Loss: 7.0755
Step 450    | Loss: 7.0755
Step 500    | Loss: 6.7958
Step 500    | Loss: 6.7958
Step 550    | Loss: 6.7924
Step 550    | Loss: 6.7924
Step 600    | Loss: 7.3102
Step 600    | Loss: 7.3102
Step 650    | Loss: 6.2226
Step 650    | Loss: 6.2226
Step 700    | Loss: 6.7447
Step 700    | Loss: 6.7447
Step 750    | Loss: 6.3326
Step 750    | Loss: 6.3326
Step 800    | Loss: 6.4986
Step 800    | Loss: 6.4986
Step 850    | Loss: 6.7022
Step 850    | Loss: 6.7022
Step 900    | Loss: 6.5472
S

In [ ]:
final_dir = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full"
os.makedirs(final_dir, exist_ok=True)

# Model + tokenizer + trainer state birlikte kaydedilir
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)
trainer.save_state()

print(f"✅ Model yeniden kaydedildi: {final_dir}")


✅ Model yeniden kaydedildi: /content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full


# 6.0 Phase 1 Test

## 6.1 Validation Loss / Perplexity

In [ ]:
import math
metrics = trainer.evaluate()
print(f"Val Loss: {metrics['eval_loss']:.4f}")
print(f"Perplexity: {math.exp(metrics['eval_loss']):.2f}")


Val Loss: 6.2360
Perplexity: 510.83


## 6.2 Language Fluency Test

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --upgrade --quiet
!pip install transformers==4.44.2 --upgrade --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 137.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 101.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_path = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full"
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

text_gen = pipeline("text-generation", model=model, tokenizer=tok)

prompt = "In the future, artificial intelligence will"
output = text_gen(prompt, max_new_tokens=80, temperature=0.8, top_p=0.9)

print("\n Model Output:\n", output[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



 Model Output:
 In the future, artificial intelligence will be the first of the first time of the first time, and the first time of the first time. The first time was the first of the first time, and the first time of the first time, and the first time of the first time. The first time was the first time in the first time, and the first time of the first time, and the first time of the first time.


## 6.3 Fluency Evaluation Test

In [ ]:
import torch, re, numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import math

# 🔹 1️⃣ Model ve tokenizer yükle
model_path = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full"
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
model.eval()

# 🔹 2️⃣ Test prompt listesi
prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "The world of tomorrow depends on",
    "Technology has changed the way we",
    "Science and creativity together can"
]

# 🔹 3️⃣ Yardımcı fonksiyonlar
def generate_text(prompt, max_new_tokens=60, temperature=0.8, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

def compute_perplexity(text):
    enc = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model(**enc, labels=enc["input_ids"])
    loss = out.loss.item()
    ppl = math.exp(loss)
    return loss, ppl

def repetition_ratio(text):
    words = re.findall(r"\b\w+\b", text.lower())
    if len(words) == 0:
        return 0
    unique_ratio = len(set(words)) / len(words)
    return 1 - unique_ratio  # yüksekse fazla tekrar var demek

# 🔹 4️⃣ Testi çalıştır
results = []
print("🚀 Running fluency test...\n")
for p in tqdm(prompts):
    gen = generate_text(p)
    loss, ppl = compute_perplexity(p + " " + gen)
    rep = repetition_ratio(gen)
    results.append((p, loss, ppl, rep, gen))
    print(f"\nPrompt: {p}\n---\n{gen}\nLoss={loss:.4f} | PPL={ppl:.2f} | Repetition={rep*100:.1f}%")

# 🔹 5️⃣ Ortalama metrikler
avg_loss = np.mean([r[1] for r in results])
avg_ppl = np.mean([r[2] for r in results])
avg_rep = np.mean([r[3] for r in results])

print("\n📊 Fluency Summary:")
print(f" • Avg Loss: {avg_loss:.4f}")
print(f" • Avg Perplexity: {avg_ppl:.2f}")
print(f" • Avg Repetition: {avg_rep*100:.1f}%")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🚀 Running fluency test...



 20%|██        | 1/5 [00:02<00:08,  2.18s/it]


Prompt: In the future, artificial intelligence will
---
be seen , with the main and were also to the time of the town . After the American , the war was still in early April , and the British government was a member of the United States , but he was the number of the show to the first season . It was still destroyed in the area
Loss=3.8323 | PPL=46.17 | Repetition=35.2%


 40%|████      | 2/5 [00:04<00:06,  2.09s/it]

...


Prompt: The world of tomorrow depends on
---
March 2014 , 2010 . The first @-@ 3 @-@ Star @-@ 3 ( 2 @.@ 5 million in ) , with the same three @-@ year @-@ class per and was a more @-@ yard @-@ American @-@ 4 @.@ 1 . The first match was the game was appointed by the first week for the film . The
Loss=3.6598 | PPL=38.85 | Repetition=25.6%


 80%|████████  | 4/5 [00:08<00:02,  2.08s/it]


Prompt: Technology has changed the way we
---
have not the British story of the first two @-@ side of the original , and they had been found to take the first time of the first , a result , and that was in the most of the end of the last day . The first time for the United States were completed , and the city of
Loss=3.6273 | PPL=37.61 | Repetition=38.9%


100%|██████████| 5/5 [00:10<00:00,  2.09s/it]


Prompt: Science and creativity together can
---
not be able to be the time of the early @-@ century of the final . The two is the second of the first time , including the next day of the film and the first two @-@ game , as the most of the American and the French . The last three @-@ point of the game is located
Loss=3.8251 | PPL=45.84 | Repetition=47.2%

📊 Fluency Summary:
 • Avg Loss: 3.7394
 • Avg Perplexity: 42.22
 • Avg Repetition: 36.3%


# 7.0 Full Training Phase 2 (0-50)

In [ ]:
# ====================================================
# 🧠 Model + Trainer yeniden yükleme (Phase 2 için)
# ====================================================

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Phase 1'de kaydettiğimiz final modeli Drive'dan alıyoruz
model_dir = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full"

tok = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype="auto", device_map="auto")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/OAIx_3B_state2_checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    warmup_steps=100,
    max_steps=20000,
    save_strategy="steps",
    save_steps=2000,
    logging_steps=50,
    report_to="none",
    fp16=True,
    bf16=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tok,
)

print("✅ Model, tokenizer ve trainer başarıyla yeniden yüklendi.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
if isinstance(trainer.args.deepspeed, dict):
    ds = trainer.args.deepspeed
else:
    ds = {}

ds["zero_optimization"] = {
    "stage": 2,
    "offload_optimizer": {"device": "cpu", "pin_memory": True},
    "offload_param": {"device": "cpu", "pin_memory": True},
}
ds["bf16"] = {"enabled": True}
ds["fp16"] = {"enabled": False}
trainer.args.deepspeed = ds


In [ ]:
# ====================================================
# ✅ Phase-2 – Weights-Only Continue (bf16, ZeRO-3, GC OFF, seq_len = 160)
#  - Phase-1 final ağırlıklarından devam eder
#  - optimizer/scheduler/scaler yüklenmez
# ====================================================

from google.colab import drive
import os, re, torch
from datasets import load_from_disk, DatasetDict
from transformers import AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainerCallback

# ====================================================
# 1️⃣ Drive bağla
# ====================================================
drive.mount("/content/drive", force_remount=True)
phase1_final = "/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full"
phase2_dir   = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints"
os.makedirs(phase2_dir, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {phase2_dir}")

# ====================================================
# 2️⃣ Dataset yükle (%50 subset)
# ====================================================
encoded_dataset = DatasetDict({
    "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
    "val":   load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
})

train_data = encoded_dataset["train"]
val_data   = encoded_dataset["val"]
train_len, val_len = len(train_data), len(val_data)

subset_train = train_data.select(range(0, int(0.50 * train_len)))
subset_val   = val_data.select(range(0, int(0.50 * val_len)))
print(f"✅ Subset hazır: {len(subset_train):,}/{train_len:,} train | {len(subset_val):,}/{val_len:,} val")

trainer.train_dataset = subset_train
trainer.eval_dataset  = subset_val

# ====================================================
# 3️⃣ Phase-1 final ağırlıklarını yükle
# ====================================================
print("🔧 Loading Phase-1 final weights (weights-only)…")
assert os.path.isdir(phase1_final), f"Phase-1 weights not found: {phase1_final}"
model = AutoModelForCausalLM.from_pretrained(
    phase1_final,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)
trainer.model = model

# ====================================================
# 4️⃣ Checkpoint sıklığı
# ====================================================
trainer.args.save_strategy = "steps"
trainer.args.save_steps = 2000          # 🔁 her 2000 adımda checkpoint
trainer.args.save_total_limit = 3       # sadece son 3 checkpoint kalsın
trainer.args.logging_steps = 50         # her 50 adımda loss göster
trainer.args.output_dir = phase2_dir
trainer.args.max_steps = 20000

# ====================================================
# 5️⃣ bf16 + TF32
# ====================================================
trainer.args.fp16 = False
trainer.args.bf16 = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try: torch.set_float32_matmul_precision("high")
except Exception: pass

# ====================================================
# 6️⃣ Collator (seq_len = 160)
# ====================================================
class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=160, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length
    def __call__(self, examples):
        batch = super().__call__(examples)
        for k in batch:
            if hasattr(batch[k], "ndim") and batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch

trainer.data_collator = TruncatingCollator(tokenizer=tok, max_length=160)

# ====================================================
# 7️⃣ ZeRO-3 + CPU offload
# ====================================================
trainer.args.deepspeed = {
    "bf16": {"enabled": True},
    "fp16": {"enabled": False},
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {"device": "cpu", "pin_memory": True},
        "offload_param": {"device": "cpu", "pin_memory": True},
        "overlap_comm": True,
        "contiguous_gradients": True,
        "stage3_max_live_parameters": 1e9,
        "stage3_prefetch_bucket_size": 5e7,
        "stage3_param_persistence_threshold": 1e6,
    },
}

# ====================================================
# 8️⃣ Gradient Checkpointing OFF
# ====================================================
try: model.gradient_checkpointing_disable()
except Exception: pass
if hasattr(model, "config"): model.config.use_cache = False

# ====================================================
# 9️⃣ Log callback
# ====================================================
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")
trainer.add_callback(LossPrinterCallback())

# ====================================================
# 🔟 Eğitimi başlat
# ====================================================
print("🚀 Starting Phase-2 (weights-only): bf16, ZeRO-3 offload, seq_len 160, bs 1×GA 4")
train_output = None
try:
    train_output = trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("❌ OOM — GA=8 veya seq_len=128 denenebilir.")
        torch.cuda.empty_cache()
        train_output = None
    else:
        raise e

# ====================================================
# 11️⃣ Final Checkpoint Kaydet
# ====================================================
if train_output is not None:
    final_dir = os.path.join(phase2_dir, "final_state2_weightsonly_len160_zero3")
    os.makedirs(final_dir, exist_ok=True)
    trainer.save_model(final_dir)
    tok.save_pretrained(final_dir)
    try: trainer.save_state()
    except Exception: pass
    print(f"✅ Final Phase-2 model saved to: {final_dir}")
else:
    print("❌ Training failed.")


Mounted at /content/drive
✅ Checkpoints will be saved to: /content/drive/MyDrive/OAIx_3B_state2_checkpoints
✅ Subset hazır: 524,263/1,048,526 train | 58,251/116,503 val
🔧 Loading Phase-1 final weights (weights-only)…


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🚀 Starting Phase-2 (weights-only): bf16, ZeRO-3 offload, seq_len 160, bs 1×GA 4


Step,Training Loss
50,5.430200
100,5.657500
150,5.478900
200,5.710400
250,5.754600
300,5.670300
350,5.642400
400,5.736500
450,5.768200
500,5.314000


Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 50     | Loss: 5.4302
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 100    | Loss: 5.6575
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 150    | Loss: 5.4789
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 200    | Loss: 5.7104
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 250    | Loss: 5.7546
Step 300    | Loss: 5.6703
Step 300    | Loss: 5.6703
S

Step,Training Loss
50,5.430200
100,5.657500
150,5.478900
200,5.710400
250,5.754600
300,5.670300
350,5.642400
400,5.736500
450,5.768200
500,5.314000


Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7500   | Loss: 5.5081
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7550   | Loss: 5.4988
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7600   | Loss: 5.4197
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7650   | Loss: 5.6630
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7700   | Loss: 5.5923
Step 7750   | Loss: 5.4451
Step 7750   | Loss: 5.4451
S

KeyboardInterrupt: 

In [ ]:
# ====================================================
# ✅ Phase-2 Resume (Colab yeniden açıldıysa)
# ====================================================
from google.colab import drive
import torch, os, re
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling
)
from datasets import load_from_disk, DatasetDict

# 1️⃣ Drive mount
drive.mount("/content/drive", force_remount=True)
ckpt_dir = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints"

# 2️⃣ Tokenizer ve dataset yükle
tok = AutoTokenizer.from_pretrained("/content/drive/MyDrive/OAIx_3B_state1_checkpoints/final_state1_full")

encoded_dataset = DatasetDict({
    "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
    "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
})

train_data = encoded_dataset["train"].select(range(0, int(0.50 * len(encoded_dataset["train"]))))
val_data   = encoded_dataset["val"].select(range(0, int(0.50 * len(encoded_dataset["val"]))))

collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

# 3️⃣ En son checkpoint’i bul
def find_latest_ckpt(base_dir):
    ckpts = [d for d in os.listdir(base_dir) if d.startswith("checkpoint-")]
    if not ckpts:
        return None
    steps = sorted([int(re.findall(r"\d+", c)[0]) for c in ckpts])
    return os.path.join(base_dir, f"checkpoint-{steps[-1]}")

latest_ckpt = find_latest_ckpt(ckpt_dir)
print(f"🔄 Found latest Phase-2 checkpoint: {latest_ckpt}")

# 4️⃣ Modeli checkpoint’ten yükle
model = AutoModelForCausalLM.from_pretrained(
    latest_ckpt,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

# 5️⃣ Training args + Trainer oluştur
training_args = TrainingArguments(
    output_dir=ckpt_dir,
    overwrite_output_dir=False,
    save_strategy="steps",
    save_steps=2000,
    save_total_limit=3,
    logging_steps=50,
    max_steps=20000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    bf16=True,
    fp16=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=collator,
    tokenizer=tok,
)

# 6️⃣ Eğitime devam et
print(f"🚀 Resuming Phase-2 training from: {latest_ckpt}")
train_output = trainer.train(resume_from_checkpoint=latest_ckpt)


Mounted at /content/drive
🔄 Found latest Phase-2 checkpoint: /content/drive/MyDrive/OAIx_3B_state2_checkpoints/checkpoint-8000


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

max_steps is given, it will override any value given in num_train_epochs


🚀 Resuming Phase-2 training from: /content/drive/MyDrive/OAIx_3B_state2_checkpoints/checkpoint-8000
[2025-10-31 12:46:46,220] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (3.4.0), only 1.0.0 is known to be compatible


Step,Training Loss
8050,5.690700
8100,5.835800
8150,5.614500
8200,5.649100
8250,5.811100
8300,5.915900
8350,5.420400
8400,6.010200
8450,5.666700
8500,5.481500


In [ ]:
# ====================================================
# ✅ Phase-2 Final Save — Full model export to Drive
# ====================================================
import os

final_dir = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full"
os.makedirs(final_dir, exist_ok=True)

# Model + tokenizer + trainer state birlikte kaydet
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)
try:
    trainer.save_state()
except Exception as e:
    print(f"⚠️ Trainer state save skipped: {e}")

print(f"\n✅ Phase-2 training completed and full model saved to:\n{final_dir}")



✅ Phase-2 training completed and full model saved to:
/content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full


# 8.0 Phase 2 Test

## 8.1 Validation Loss / Perplexity

In [ ]:
# ====================================================
# ✅ Validation Loss + Perplexity
# ====================================================
import math
from transformers import Trainer

# Validation seti yeniden bağla (eğer trainer eval_dataset yoksa)
if not hasattr(trainer, "eval_dataset") or trainer.eval_dataset is None:
    from datasets import load_from_disk, DatasetDict
    encoded_dataset = DatasetDict({
        "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
        "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
    })
    val_data = encoded_dataset["val"].select(range(0, int(0.50 * len(encoded_dataset["val"]))))
    trainer.eval_dataset = val_data

# Evaluate
eval_results = trainer.evaluate()
val_loss = eval_results["eval_loss"]
perplexity = math.exp(val_loss)

print(f"\n📊 Validation Results:")
print(f" • Val Loss: {val_loss:.4f}")
print(f" • Perplexity: {perplexity:.2f}")



📊 Validation Results:
 • Val Loss: 6.1920
 • Perplexity: 488.84


## 8.2 Language Fluency Test

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --upgrade --quiet
!pip install transformers==4.44.2 --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 143.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ✅ Phase-2 final model yolunu kullan
model_path = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full"

tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

text_gen = pipeline("text-generation", model=model, tokenizer=tok)

prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "Technology has changed the way we",
    "The purpose of creativity is to",
    "If machines could dream, they would"
]

for p in prompts:
    print(f"\n🧩 Prompt: {p}")
    out = text_gen(p, max_new_tokens=80, temperature=0.8, top_p=0.9, do_sample=True)
    print(out[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


🧩 Prompt: In the future, artificial intelligence will
In the future, artificial intelligence will be the state of the first game, the series, who had not to be one. They were still at the following year, and, as they had been to be a number of the next day. This is an end of the final and the next day. A small release of the city, the latter is no @-@ based on the series, but the latter is now in the episode's

🧩 Prompt: Humans and AI will work together to
Humans and AI will work together to be the series, the city of the latter of the series and other of the two @-@ up to play. The film's first to the latter was the ". However, it was still known as a " song's ], the song was first to be written by a single, and that it was a " good album ". The album was a year of the new part of the

🧩 Prompt: Technology has changed the way we
Technology has changed the way we, which is the same in the episode of the show of the first season, and the United States is an open with an album to be a w

## 8.3 Fluency Evaluation Test

In [ ]:
import torch, re, numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import math

# 🔹 1️⃣ Model ve tokenizer yükle
model_path = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full"
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
model.eval()

# 🔹 2️⃣ Test prompt listesi
prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "The world of tomorrow depends on",
    "Technology has changed the way we",
    "Science and creativity together can"
]

# 🔹 3️⃣ Yardımcı fonksiyonlar
def generate_text(prompt, max_new_tokens=60, temperature=0.8, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

def compute_perplexity(text):
    enc = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model(**enc, labels=enc["input_ids"])
    loss = out.loss.item()
    ppl = math.exp(loss)
    return loss, ppl

def repetition_ratio(text):
    words = re.findall(r"\b\w+\b", text.lower())
    if len(words) == 0:
        return 0
    unique_ratio = len(set(words)) / len(words)
    return 1 - unique_ratio  # yüksekse fazla tekrar var demek

# 🔹 4️⃣ Testi çalıştır
results = []
print("🚀 Running fluency test...\n")
for p in tqdm(prompts):
    gen = generate_text(p)
    loss, ppl = compute_perplexity(p + " " + gen)
    rep = repetition_ratio(gen)
    results.append((p, loss, ppl, rep, gen))
    print(f"\nPrompt: {p}\n---\n{gen}\nLoss={loss:.4f} | PPL={ppl:.2f} | Repetition={rep*100:.1f}%")

# 🔹 5️⃣ Ortalama metrikler
avg_loss = np.mean([r[1] for r in results])
avg_ppl = np.mean([r[2] for r in results])
avg_rep = np.mean([r[3] for r in results])

print("\n📊 Fluency Summary:")
print(f" • Avg Loss: {avg_loss:.4f}")
print(f" • Avg Perplexity: {avg_ppl:.2f}")
print(f" • Avg Repetition: {avg_rep*100:.1f}%")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 Running fluency test...



 20%|██        | 1/5 [00:02<00:08,  2.24s/it]


Prompt: In the future, artificial intelligence will
---
have been created in the storm . It has been shown with a small @-@ year @-@ up @-@ selling and @-@ off @-@ west of the main world of the city , but the song is the next side . The first two @-@ based in the state 's new name to the first point , and is
Loss=3.9298 | PPL=50.90 | Repetition=26.5%


 40%|████      | 2/5 [00:04<00:06,  2.20s/it]

...


Prompt: The world of tomorrow depends on
---
the United States , the second and the main @-@ day in the German Empire in the British Army , the new Catholic and the city , and the first time . It was first made by the end of the final @-@ up @-@ year @-@ class on the first year , with the other period of the United
Loss=3.7378 | PPL=42.01 | Repetition=39.2%


 80%|████████  | 4/5 [00:08<00:02,  2.18s/it]


Prompt: Technology has changed the way we
---
has been as a small @-@ side , and the first one number of the film 's parents . The new episode was not a man , the following single to the new release . The most of the same time , the world and the first episode , was awarded by the season . The episode was to
Loss=3.6483 | PPL=38.41 | Repetition=39.2%


100%|██████████| 5/5 [00:10<00:00,  2.20s/it]


Prompt: Science and creativity together can
---
be a localable with the last time . The first was the first round for the first time , but he is an important longer @-@ like force . In a single , he was not a single for the time : " The film , a lot of a " . " The series has been the last
Loss=3.7904 | PPL=44.27 | Repetition=41.7%

📊 Fluency Summary:
 • Avg Loss: 3.7998
 • Avg Perplexity: 44.92
 • Avg Repetition: 36.0%


# 9.0 Full Training Phase 3 (0-75)

In [ ]:
# ====================================================
# ✅ Phase-3 Training – 0–75 % dataset, seq_len=768, 30 000 steps
# ====================================================

from google.colab import drive
import os, re, torch
from datasets import DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, TrainerCallback
)

# ---- 1️⃣ Drive bağla
drive.mount("/content/drive", force_remount=True)
drive_ckpt_dir = "/content/drive/MyDrive/OAIx_3B_state3_checkpoints"
os.makedirs(drive_ckpt_dir, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {drive_ckpt_dir}")

# ---- 2️⃣ Dataset yükle (%75 subset)
encoded_dataset = DatasetDict({
    "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
    "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
})
train_data = encoded_dataset["train"]
val_data   = encoded_dataset["val"]

train_len = len(train_data)
val_len   = len(val_data)

subset_train = train_data.select(range(0, int(0.75 * train_len)))
subset_val   = val_data.select(range(0, int(0.75 * val_len)))
print(f"✅ Subset ready: {len(subset_train):,}/{train_len:,} train | {len(subset_val):,}/{val_len:,} val")

# ---- 3️⃣ Phase-2 ağırlıklarını yükle
phase2_final = "/content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full"
assert os.path.isdir(phase2_final), f" Phase-2 final weights bulunamadı: {phase2_final}"

print(f" Loading Phase-2 final weights from: {phase2_final}")
tok = AutoTokenizer.from_pretrained(phase2_final)
model = AutoModelForCausalLM.from_pretrained(
    phase2_final,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

# ---- 4️⃣ Collator (seq_len = 768)
class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=768, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length
    def __call__(self, examples):
        batch = super().__call__(examples)
        for k in batch:
            if hasattr(batch[k], "ndim") and batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch
collator = TruncatingCollator(tok, max_length=768)

# ---- 5️⃣ TrainingArguments
training_args = TrainingArguments(
    output_dir=drive_ckpt_dir,
    overwrite_output_dir=True,
    save_strategy="steps",
    save_steps=2000,
    save_total_limit=3,
    logging_steps=50,
    num_train_epochs=1,
    max_steps=30000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    bf16=True,
    report_to="none"
)

# ---- 6️⃣ Trainer oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=subset_train,
    eval_dataset=subset_val,
    tokenizer=tok
)

# ---- 7️⃣ Log callback
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")
trainer.add_callback(LossPrinterCallback())

# ---- 8️⃣ GPU performans ayarları
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ---- 9️⃣ Eğitimi başlat
try:
    print(" Starting Phase-3 Training (0–75 %, seq_len=768, 30k steps)…")
    train_output = trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print(" OOM detected — batch veya seq_len düşürülmeli.")
        torch.cuda.empty_cache()
        train_output = None
    else:
        raise e

# ---- 🔟 Final modeli kaydet
if train_output is not None:
    final_dir = os.path.join(drive_ckpt_dir, "final_state3_full_len768")
    os.makedirs(final_dir, exist_ok=True)
    trainer.save_model(final_dir)
    tok.save_pretrained(final_dir)
    try:
        trainer.save_state()
    except Exception:
        pass
    print(f" Final Phase-3 model saved to: {final_dir}")
else:
    print(" Training failed.")


Mounted at /content/drive
✅ Checkpoints will be saved to: /content/drive/MyDrive/OAIx_3B_state3_checkpoints
✅ Subset ready: 786,394/1,048,526 train | 87,377/116,503 val
 Loading Phase-2 final weights from: /content/drive/MyDrive/OAIx_3B_state2_checkpoints/final_state2_full


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

max_steps is given, it will override any value given in num_train_epochs


 Starting Phase-3 Training (0–75 %, seq_len=768, 30k steps)…


Step,Training Loss
50,5.743900
100,5.676000
150,5.622600
200,5.605500
250,5.745300
300,5.748400
350,5.620200
400,5.500600
450,5.922300
500,5.570100


Step 50     | Loss: 5.7439
Step 100    | Loss: 5.6760
Step 150    | Loss: 5.6226
Step 200    | Loss: 5.6055
Step 250    | Loss: 5.7453
Step 300    | Loss: 5.7484
Step 350    | Loss: 5.6202
Step 400    | Loss: 5.5006
Step 450    | Loss: 5.9223
Step 500    | Loss: 5.5701
Step 550    | Loss: 5.6906
Step 600    | Loss: 5.7923
Step 650    | Loss: 5.3229
Step 700    | Loss: 5.5088
Step 750    | Loss: 5.6797
Step 800    | Loss: 5.3178
Step 850    | Loss: 5.5599
Step 900    | Loss: 5.4098
Step 950    | Loss: 5.6327
Step 1000   | Loss: 5.6407
Step 1050   | Loss: 5.5413
Step 1100   | Loss: 5.5164
Step 1150   | Loss: 5.3367
Step 1200   | Loss: 5.3028
Step 1250   | Loss: 5.4370
Step 1300   | Loss: 5.5331
Step 1350   | Loss: 5.5075
Step 1400   | Loss: 5.5539
Step 1450   | Loss: 5.4035
Step 1500   | Loss: 5.5487
Step 1550   | Loss: 5.3313
Step 1600   | Loss: 5.5517
Step 1650   | Loss: 5.4694
Step 1700   | Loss: 5.5377
Step 1750   | Loss: 5.6235
Step 1800   | Loss: 5.4855
Step 1850   | Loss: 5.4332
S

eğitim bittekten sonra bu kodu çalıştır. Bu kod google drive a kayıt yapıyor. orada OAIx_3B_state3_checkpoints dosyası göreceksin, o dosyanın içinda final_state3_full dosyası olacak. orayı kontrol et. Eğitim belgeleri oraya kaydolacak.

In [ ]:
# ====================================================
# ✅ Phase-3 Final Save — Full model export to Drive
# ====================================================
import os

final_dir = "/content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full"
os.makedirs(final_dir, exist_ok=True)

# Model + tokenizer + trainer state birlikte kaydet
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)

try:
    trainer.save_state()
except Exception as e:
    print(f"⚠️ Trainer state save skipped: {e}")

print(f"\n✅ Phase-3 training completed and full model saved to:\n{final_dir}")



✅ Phase-3 training completed and full model saved to:
/content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full


Kontrol edeceğin belgelerde sırayla şunlar olmalı:


*   config.json
*   generation_config.json
*   model-00001-of-00002.safetensors
*   model-00001-of-00002.safetensors
*   model.safetensors.index.json
*   special_tokens_map.json
*   tokenizer_config.json
*   tokenizer.json
*   tokenizer.model
*   training_args.bin







# 10 Phase 3 Test

## 10.1 Validation Loss / Perplexity

Bu eğitim kodunu çalıştır yaklaşık 30 - 40 dk sürer. Çıktıyı bana at

In [ ]:
# ====================================================
# ✅ Validation Loss + Perplexity
# ====================================================
import math
from transformers import Trainer

# Validation seti yeniden bağla (eğer trainer eval_dataset yoksa)
if not hasattr(trainer, "eval_dataset") or trainer.eval_dataset is None:
    from datasets import load_from_disk, DatasetDict
    encoded_dataset = DatasetDict({
        "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
        "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
    })
    val_data = encoded_dataset["val"].select(range(0, int(0.50 * len(encoded_dataset["val"]))))
    trainer.eval_dataset = val_data

# Evaluate
eval_results = trainer.evaluate()
val_loss = eval_results["eval_loss"]
perplexity = math.exp(val_loss)

print(f"\n📊 Validation Results:")
print(f" • Val Loss: {val_loss:.4f}")
print(f" • Perplexity: {perplexity:.2f}")



📊 Validation Results:
 • Val Loss: 5.5738
 • Perplexity: 263.43


## 10.2 Language Fluency Test

Bu kütüphane kodunu çalıştır yüklendikten sonra yukarıda bulunan Runtime sekmesine tıkla orada Restart section kısmını göreceksin ona tıkla.

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --upgrade --quiet
!pip install transformers==4.44.2 --upgrade --quiet

 Daha sonra altındaki kodu çalıştır bu da farklı bir eğitim.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ✅ Phase-2 final model yolunu kullan
model_path = "/content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full"

tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

text_gen = pipeline("text-generation", model=model, tokenizer=tok)

prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "Technology has changed the way we",
    "The purpose of creativity is to",
    "If machines could dream, they would"
]

for p in prompts:
    print(f"\n🧩 Prompt: {p}")
    out = text_gen(p, max_new_tokens=80, temperature=0.8, top_p=0.9, do_sample=True)
    print(out[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


🧩 Prompt: In the future, artificial intelligence will
In the future, artificial intelligence will be taken to the city and the city in the world. As the castle is often required to be an example of a number of economic and body, the building was a small number of the more evidence of the land. The city has been a distinct and white, while a common problem, that can cause a single factor on the. The planet is that they are a very common. The average

🧩 Prompt: Humans and AI will work together to
Humans and AI will work together to make the game's release : " That's ] ever criticized on the plot. " As the album's " most of the most memorable. " I Am It's " Who " is a great, I am it, and a story, was never well received. " After the episode, " I was very difficult ", I think it's not only a few months.

🧩 Prompt: Technology has changed the way we
Technology has changed the way we did not have any one of his most important and a man of a " unaffected ". In the early 1930s, the company bec

## 10.3 Fluency Evaluation Test

En son bu eğitim kodunu çalıştır ve bana çıktıları whatsapptan at. Seni seviyoree. Bütün her şey bittiğinde ben gelmiş olurum veya gelmek üzereyimdir.

In [ ]:
import torch, re, numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import math

# 🔹 1️⃣ Model ve tokenizer yükle
model_path = "/content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full"
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
model.eval()

# 🔹 2️⃣ Test prompt listesi
prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "The world of tomorrow depends on",
    "Technology has changed the way we",
    "Science and creativity together can"
]

# 🔹 3️⃣ Yardımcı fonksiyonlar
def generate_text(prompt, max_new_tokens=60, temperature=0.8, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

def compute_perplexity(text):
    enc = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model(**enc, labels=enc["input_ids"])
    loss = out.loss.item()
    ppl = math.exp(loss)
    return loss, ppl

def repetition_ratio(text):
    words = re.findall(r"\b\w+\b", text.lower())
    if len(words) == 0:
        return 0
    unique_ratio = len(set(words)) / len(words)
    return 1 - unique_ratio  # yüksekse fazla tekrar var demek

# 🔹 4️⃣ Testi çalıştır
results = []
print("🚀 Running fluency test...\n")
for p in tqdm(prompts):
    gen = generate_text(p)
    loss, ppl = compute_perplexity(p + " " + gen)
    rep = repetition_ratio(gen)
    results.append((p, loss, ppl, rep, gen))
    print(f"\nPrompt: {p}\n---\n{gen}\nLoss={loss:.4f} | PPL={ppl:.2f} | Repetition={rep*100:.1f}%")

# 🔹 5️⃣ Ortalama metrikler
avg_loss = np.mean([r[1] for r in results])
avg_ppl = np.mean([r[2] for r in results])
avg_rep = np.mean([r[3] for r in results])

print("\n📊 Fluency Summary:")
print(f" • Avg Loss: {avg_loss:.4f}")
print(f" • Avg Perplexity: {avg_ppl:.2f}")
print(f" • Avg Repetition: {avg_rep*100:.1f}%")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 Running fluency test...



 20%|██        | 1/5 [00:02<00:09,  2.27s/it]


Prompt: In the future, artificial intelligence will
---
be used by the Great Western Association , the French Government was not a state , and the only case was not on the island of the French government . In the early 1990s the ship was given the same name , while the French had the ship 's main event of the county . The storm was
Loss=3.6268 | PPL=37.59 | Repetition=37.7%


 40%|████      | 2/5 [00:04<00:06,  2.24s/it]

...


Prompt: The world of tomorrow depends on
---
February 9 , 2008 , but the following day was used for the following day on the British side , and the British forces was to play the French , and the British had begun . The ship was completed by the Royal Navy on August 6 , 2002 . The brigade had been the only major American ship at
Loss=3.3698 | PPL=29.07 | Repetition=35.8%


 80%|████████  | 4/5 [00:08<00:02,  2.22s/it]


Prompt: Technology has changed the way we
---
have been done . If this , the people 's most likely and in the game 's book The King 's book , " the most likely in the story and the most recent songs that they are " . While the music was the most successful in the same time , the album was not a
Loss=3.4668 | PPL=32.03 | Repetition=40.0%


100%|██████████| 5/5 [00:11<00:00,  2.22s/it]


Prompt: Science and creativity together can
---
be considered to their own and power . It is now very more than the time of a " mubity " , which is also the " most important " of the city , which are most common with a number of the " rock " ( " ) , which is the largest or " black "
Loss=3.4900 | PPL=32.79 | Repetition=27.9%

📊 Fluency Summary:
 • Avg Loss: 3.4340
 • Avg Perplexity: 31.29
 • Avg Repetition: 35.6%


# 11 Full Training Phase 4 (0-100)

In [ ]:
# ====================================================
# ✅ Phase-4 — Full Dataset (seq_len=2048, lr=2e-6, 40k steps)
# ====================================================

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, TrainerCallback
)
from datasets import load_from_disk
from google.colab import drive
import torch, os

# ---- 1️⃣ Mount & dirs
drive.mount("/content/drive", force_remount=True)
phase3_final = "/content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full"
phase4_ckpt  = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints"
os.makedirs(phase4_ckpt, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {phase4_ckpt}")

# ---- 2️⃣ Dataset (full 0–100 %)
train_data = load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train")
val_data   = load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
print(f"✅ Full dataset loaded: {len(train_data):,} train | {len(val_data):,} val")

# ---- 3️⃣ Load Phase-3 model
print(f"🔄 Loading Phase-3 final model from: {phase3_final}")
tok = AutoTokenizer.from_pretrained(phase3_final)
model = AutoModelForCausalLM.from_pretrained(
    phase3_final,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
).to("cuda")
print("✅ Model fully loaded on GPU (bf16, seq_len=2048)")

# ---- 4️⃣ Collator (truncate 2048)
class TruncatingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length=2048, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, **kwargs)
        self.max_length = max_length
    def __call__(self, examples):
        batch = super().__call__(examples)
        for k in batch:
            if hasattr(batch[k], "ndim") and batch[k].ndim == 2:
                batch[k] = batch[k][:, :self.max_length]
        return batch
collator = TruncatingCollator(tok, max_length=2048)

# ---- 5️⃣ TrainingArgs
training_args = TrainingArguments(
    output_dir=phase4_ckpt,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-6,
    num_train_epochs=1,
    max_steps=40000,
    save_strategy="steps",
    save_steps=10000,
    save_total_limit=2,
    logging_steps=50,
    bf16=True,
    report_to="none",
)

# ---- 6️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tok,
)

# ---- 7️⃣ Reset optimizer/scheduler
trainer.optimizer = None
trainer.lr_scheduler = None
print("🧹 Optimizer & scheduler reset (clean start).")

# ---- 8️⃣ Loss callback
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")
trainer.add_callback(LossPrinterCallback())

# ---- 9️⃣ Train
print("🚀 Starting Phase-4 (seq_len=2048, lr=2e-6, 40k steps)...")
train_output = trainer.train()

# ---- 🔟 Final save
final_dir = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full"
os.makedirs(final_dir, exist_ok=True)
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)
try:
    trainer.save_state()
except Exception:
    pass
print(f"✅ Final model saved to: {final_dir}")


Mounted at /content/drive
✅ Checkpoints will be saved to: /content/drive/MyDrive/OAIx_3B_state4_checkpoints
✅ Full dataset loaded: 1,048,526 train | 116,503 val
🔄 Loading Phase-3 final model from: /content/drive/MyDrive/OAIx_3B_state3_checkpoints/final_state3_full


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

max_steps is given, it will override any value given in num_train_epochs


✅ Model fully loaded on GPU (bf16, seq_len=2048)
🧹 Optimizer & scheduler reset (clean start).
🚀 Starting Phase-4 (seq_len=2048, lr=2e-6, 40k steps)...


Step,Training Loss
50,4.768300
100,4.928000
150,4.957200
200,4.884700
250,4.702600
300,5.197400
350,4.916900
400,4.877100
450,4.912200
500,4.773400


Step 50     | Loss: 4.7683
Step 100    | Loss: 4.9280
Step 150    | Loss: 4.9572
Step 200    | Loss: 4.8847
Step 250    | Loss: 4.7026
Step 300    | Loss: 5.1974
Step 350    | Loss: 4.9169
Step 400    | Loss: 4.8771
Step 450    | Loss: 4.9122
Step 500    | Loss: 4.7734
Step 550    | Loss: 5.0633
Step 600    | Loss: 4.8895
Step 650    | Loss: 5.0225
Step 700    | Loss: 4.8257
Step 750    | Loss: 5.0307
Step 800    | Loss: 4.9666
Step 850    | Loss: 4.8879
Step 900    | Loss: 4.9613
Step 950    | Loss: 5.1498
Step 1000   | Loss: 4.9623
Step 1050   | Loss: 4.8388
Step 1100   | Loss: 5.1270
Step 1150   | Loss: 4.9083
Step 1200   | Loss: 4.9122
Step 1250   | Loss: 4.9231
Step 1300   | Loss: 4.8991
Step 1350   | Loss: 5.0890
Step 1400   | Loss: 5.1122
Step 1450   | Loss: 5.0246
Step 1500   | Loss: 4.7972
Step 1550   | Loss: 4.9397
Step 1600   | Loss: 4.8906
Step 1650   | Loss: 5.0781
Step 1700   | Loss: 5.0207
Step 1750   | Loss: 4.9228
Step 1800   | Loss: 4.9541
Step 1850   | Loss: 5.1325
S

# 12 Phase 4 Test

## 12.1 Validation Loss / Perplexity

In [ ]:
# ====================================================
# ✅ Validation Loss + Perplexity
# ====================================================
import math
from transformers import Trainer

# Validation seti yeniden bağla (eğer trainer eval_dataset yoksa)
if not hasattr(trainer, "eval_dataset") or trainer.eval_dataset is None:
    from datasets import load_from_disk, DatasetDict
    encoded_dataset = DatasetDict({
        "train": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/train"),
        "val": load_from_disk("/content/oaix-3b/data/encoded/len512_fast_tok32000_fixed/val")
    })
    val_data = encoded_dataset["val"].select(range(0, int(0.50 * len(encoded_dataset["val"]))))
    trainer.eval_dataset = val_data

# Evaluate
eval_results = trainer.evaluate()
val_loss = eval_results["eval_loss"]
perplexity = math.exp(val_loss)

print(f"\n📊 Validation Results:")
print(f" • Val Loss: {val_loss:.4f}")
print(f" • Perplexity: {perplexity:.2f}")



📊 Validation Results:
 • Val Loss: 5.5729
 • Perplexity: 263.21


## 12.2 Language Fluency Test

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --upgrade --quiet
!pip install transformers==4.44.2 --upgrade --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ✅ Phase-2 final model yolunu kullan
model_path = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full"

tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

text_gen = pipeline("text-generation", model=model, tokenizer=tok)

prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "Technology has changed the way we",
    "The purpose of creativity is to",
    "If machines could dream, they would"
]

for p in prompts:
    print(f"\n🧩 Prompt: {p}")
    out = text_gen(p, max_new_tokens=80, temperature=0.8, top_p=0.9, do_sample=True)
    print(out[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


🧩 Prompt: In the future, artificial intelligence will
In the future, artificial intelligence will be used by a new number of water and a large number of the surface and a similar @-@ type of the surface of a human star. The average of the large number of a large or small number of the most important of the main species in the country and the city's population. This is the only one of the site and is the first to a few distance, and that is a greater

🧩 Prompt: Humans and AI will work together to
Humans and AI will work together to the United States, but this was the only one. However, as it was a member of the town of the Year, a member of the Year, and the " French " was awarded a " 10 @.@ 2 " in a new series. In the second quarter, the main station had been held at the same time and the third and two minutes were still in the second half of the first.

🧩 Prompt: Technology has changed the way we
Technology has changed the way we had to be a huge role for all time. If some of the gam

## 12.3 Fluency Evaluation Test

In [ ]:
import torch, re, numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import math

# 🔹 1️⃣ Model ve tokenizer yükle
model_path = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full"
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
model.eval()

# 🔹 2️⃣ Test prompt listesi
prompts = [
    "In the future, artificial intelligence will",
    "Humans and AI will work together to",
    "The world of tomorrow depends on",
    "Technology has changed the way we",
    "Science and creativity together can"
]

# 🔹 3️⃣ Yardımcı fonksiyonlar
def generate_text(prompt, max_new_tokens=60, temperature=0.8, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

def compute_perplexity(text):
    enc = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model(**enc, labels=enc["input_ids"])
    loss = out.loss.item()
    ppl = math.exp(loss)
    return loss, ppl

def repetition_ratio(text):
    words = re.findall(r"\b\w+\b", text.lower())
    if len(words) == 0:
        return 0
    unique_ratio = len(set(words)) / len(words)
    return 1 - unique_ratio  # yüksekse fazla tekrar var demek

# 🔹 4️⃣ Testi çalıştır
results = []
print("🚀 Running fluency test...\n")
for p in tqdm(prompts):
    gen = generate_text(p)
    loss, ppl = compute_perplexity(p + " " + gen)
    rep = repetition_ratio(gen)
    results.append((p, loss, ppl, rep, gen))
    print(f"\nPrompt: {p}\n---\n{gen}\nLoss={loss:.4f} | PPL={ppl:.2f} | Repetition={rep*100:.1f}%")

# 🔹 5️⃣ Ortalama metrikler
avg_loss = np.mean([r[1] for r in results])
avg_ppl = np.mean([r[2] for r in results])
avg_rep = np.mean([r[3] for r in results])

print("\n📊 Fluency Summary:")
print(f" • Avg Loss: {avg_loss:.4f}")
print(f" • Avg Perplexity: {avg_ppl:.2f}")
print(f" • Avg Repetition: {avg_rep*100:.1f}%")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 Running fluency test...



 20%|██        | 1/5 [00:02<00:09,  2.26s/it]


Prompt: In the future, artificial intelligence will
---
be found . The name is not to be used for a very part of the population in the world . It was used for the first time in the United States on November 23 , 2006 . The original name was released in August 1945 to be the first major version of the United States . This is the
Loss=3.1575 | PPL=23.51 | Repetition=40.0%


 40%|████      | 2/5 [00:04<00:06,  2.22s/it]

...


Prompt: The world of tomorrow depends on
---
the first half of the season , in a 2 – 0 victory , and a record of the year , had a 3 – 0 lead to 5 @.@ 1 billion . The season is the third and episode 's highest @-@ year @-@ off game in the season . The release , it reached number four on
Loss=2.9499 | PPL=19.10 | Repetition=31.2%


 80%|████████  | 4/5 [00:08<00:02,  2.20s/it]


Prompt: Technology has changed the way we
---
can be a " little @-@ scale " . In a period of a new group , the book must be used as a good , or a long , and the most notable man , and the book , that can be a " big and fun " . The first person was seen by the first half of
Loss=3.6010 | PPL=36.63 | Repetition=35.4%


100%|██████████| 5/5 [00:10<00:00,  2.19s/it]


Prompt: Science and creativity together can
---
be adapted to any other than the same name of the world , and that it would not be a number of a life that will be used in the development of the film . The episode is seen by The Beatles , and the album 's character was to be a single @-@ up for the " Big
Loss=3.2862 | PPL=26.74 | Repetition=31.5%

📊 Fluency Summary:
 • Avg Loss: 3.3665
 • Avg Perplexity: 30.48
 • Avg Repetition: 35.6%


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 🔹 Phase-4 final model yolunu ver
model_path = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full"

# 🔹 Modeli yükle
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")

# 🔹 Basit text-generation pipeline’ı kur
chat = pipeline("text-generation", model=model, tokenizer=tok)

# 🔹 Test cümleleri
prompts = [
    "Hi, how are you?",
    "Hello!",
    "Good morning, how are you feeling today?",
    "What is your name?",
    "Tell me something funny.",
]

# 🔹 Her prompt için cevap üret
for p in prompts:
    out = chat(p, max_new_tokens=80, temperature=0.8, top_p=0.9)
    print(f"\n🧩 Prompt: {p}\n---\n{out[0]['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



🧩 Prompt: Hi, how are you?
---
Hi, how are you?, the first of the most important and the most important, and the most important, and the most important, and the most important, the most important, and the most important, and the most important, and the most important. The game's first @-@ time @-@ known @-@ known, and the game's most important and the game's most important and the game's most important.

🧩 Prompt: Hello!
---
Hello!, the first of the first two @-@ year @-@ old season, was the first time in the season. The game was the first time in the United States, and the second season was the first time in the United States. The game was the first time in the United States, and the second season was the second season. The game was the first time in the United States, and the second season

🧩 Prompt: Good morning, how are you feeling today?
---
Good morning, how are you feeling today?, and the player's first @-@ time @-@ up. The game's first @-@ time @-@ up was released on the Billb

# 13 Phase 5 Chat Instruction Supervized Fine-Tuned

In [11]:
# =====================================================
# ✅ Phase-7 — Chat Fine-Tuning (User–Assistant format)
# =====================================================

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, TrainerCallback
)
from datasets import load_dataset, concatenate_datasets
from google.colab import drive
import torch, os

# ---- 1️⃣ Mount Drive & paths
drive.mount("/content/drive", force_remount=True)

base_model_path = "/content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full"
save_dir = "/content/drive/MyDrive/OAIx_3B_state7_ChatSFT"
os.makedirs(save_dir, exist_ok=True)
print(f"✅ Base: {base_model_path}")
print(f"✅ Checkpoints → {save_dir}")

# ---- 2️⃣ Load pure chat datasets (user–assistant)
oasst = load_dataset("OpenAssistant/oasst1", split="train[:15000]")
ultrachat = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft[:15000]")
openorca = load_dataset("Open-Orca/OpenOrca", split="train[:15000]")

# ---- 3️⃣ Convert to User–Assistant pairs
def unify(example):
    instr = (
        example.get("instruction")
        or example.get("input")
        or example.get("system_prompt")
        or example.get("prompt")
        or ""
    )
    resp = (
        example.get("output")
        or example.get("response")
        or example.get("text")
        or ""
    )
    if not instr.strip() or not resp.strip():
        return {"text": None}
    return {"text": f"User: {instr.strip()}\nAssistant: {resp.strip()}"}

oasst = oasst.map(unify)
ultrachat = ultrachat.map(unify)
openorca = openorca.map(unify)

oasst = oasst.filter(lambda x: x["text"] is not None)
ultrachat = ultrachat.filter(lambda x: x["text"] is not None)
openorca = openorca.filter(lambda x: x["text"] is not None)

dataset = concatenate_datasets([oasst, ultrachat, openorca]).shuffle(seed=42)
dataset = dataset.filter(lambda x: len(x["text"].split()) < 800)
dataset = dataset.train_test_split(test_size=0.02)
print(f"✅ Chat dataset ready: {len(dataset['train']):,} train | {len(dataset['test']):,} val")

# ---- 4️⃣ Tokenizer & model
tok = AutoTokenizer.from_pretrained(base_model_path, use_fast=True)
tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
).to("cuda")

print("✅ Model loaded and ready for chat SFT.")

# ---- 5️⃣ Tokenization with masking (loss only on assistant)
def tokenize_mask(batch):
    inputs = []
    labels = []
    for text in batch["text"]:
        if "Assistant:" not in text:
            continue
        parts = text.split("Assistant:")
        user_part = parts[0] + "Assistant:"
        asst_part = parts[1]

        full = user_part + asst_part
        tok_data = tok(full, truncation=True, max_length=1024, padding="max_length")
        labels_ = tok_data["input_ids"].copy()

        # mask out everything before assistant response
        user_len = len(tok(user_part)["input_ids"]) - 1
        labels_[:user_len] = [-100] * user_len

        inputs.append(tok_data["input_ids"])
        labels.append(labels_)

    return {"input_ids": inputs, "labels": labels, "attention_mask": [ [1]*len(x) for x in inputs ]}

tokenized = dataset.map(tokenize_mask, batched=True, remove_columns=["text"])
print("✅ Tokenization + masking complete.")

# ---- 6️⃣ Collator
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# ---- 7️⃣ Training configuration
training_args = TrainingArguments(
    output_dir=save_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    max_steps=8000,
    warmup_ratio=0.05,
    weight_decay=0.01,
    bf16=True,
    save_strategy="steps",
    save_steps=4000,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=2000,
    save_total_limit=2,
    report_to="none",
)

# ---- 8️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tok,
)

# ---- 9️⃣ Loss logger
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")
trainer.add_callback(LossPrinterCallback())

# ---- 🔟 Start training
print("🚀 Starting Chat Fine-Tuning (Phase-7)...")
trainer.train()

# ---- 🔚 Save model
final_dir = f"{save_dir}/final_chat"
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)
print(f"✅ Final Chat Model saved to {final_dir}")


Mounted at /content/drive
✅ Base: /content/drive/MyDrive/OAIx_3B_state4_checkpoints/final_state4_full
✅ Checkpoints → /content/drive/MyDrive/OAIx_3B_state7_ChatSFT


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


✅ Tokenization + masking complete.
🚀 Starting Chat Fine-Tuning (Phase-7)...


Step,Training Loss,Validation Loss
2000,4.562100,4.969237
4000,4.349700,4.852820
6000,4.471700,4.835369
8000,4.436000,4.832982


Step 50     | Loss: 12.6314
Step 100    | Loss: 10.4044
Step 150    | Loss: 8.2974
Step 200    | Loss: 7.5463
Step 250    | Loss: 7.2596
Step 300    | Loss: 6.9401
Step 350    | Loss: 6.6303
Step 400    | Loss: 6.1614
Step 450    | Loss: 5.8100
Step 500    | Loss: 5.5682
Step 550    | Loss: 5.4323
Step 600    | Loss: 5.3117
Step 650    | Loss: 5.1552
Step 700    | Loss: 5.1868
Step 750    | Loss: 5.1446
Step 800    | Loss: 5.0953
Step 850    | Loss: 5.0664
Step 900    | Loss: 4.8958
Step 950    | Loss: 4.9308
Step 1000   | Loss: 4.9312
Step 1050   | Loss: 4.9108
Step 1100   | Loss: 4.6559
Step 1150   | Loss: 4.9037
Step 1200   | Loss: 4.7794
Step 1250   | Loss: 4.7230
Step 1300   | Loss: 4.7424
Step 1350   | Loss: 4.6192
Step 1400   | Loss: 4.7570
Step 1450   | Loss: 4.6975
Step 1500   | Loss: 4.5841
Step 1550   | Loss: 4.6450
Step 1600   | Loss: 4.6159
Step 1650   | Loss: 4.7201
Step 1700   | Loss: 4.7373
Step 1750   | Loss: 4.5762
Step 1800   | Loss: 4.5864
Step 1850   | Loss: 4.4668

In [12]:
# ====================================================
# ✅ Phase-5 Chat Model Evaluation
# ====================================================

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ---- 1️⃣ Paths
model_path = "/content/drive/MyDrive/OAIx_3B_state7_ChatSFT/final_chat"  # eğitimin kaydedildiği klasör

# ---- 2️⃣ Load model
tok = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
).to("cuda")

print("✅ Model loaded successfully!\n")

# ---- 3️⃣ Helper to chat
def chat(prompt, max_new_tokens=200):
    formatted = f"### Instruction:\n{prompt.strip()}\n\n### Response:\n"
    inputs = tok(formatted, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.8, top_p=0.9, do_sample=True)
    response = tok.decode(outputs[0], skip_special_tokens=True)
    response = response.split("### Response:")[-1].strip()
    print(f"💭 Prompt: {prompt}")
    print(f"💬 Response: {response}\n{'-'*80}\n")

# ---- 4️⃣ Test prompts
tests = [
    "Hi, how are you today?",
    "Tell me a fun fact about space.",
    "If you could travel anywhere, where would you go?",
    "Explain photosynthesis in simple terms.",
    "Tell me a short motivational quote."
]

for t in tests:
    chat(t)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Model loaded successfully!



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


💭 Prompt: Hi, how are you today?
💬 Response: 's first album liner notes . She also has been written with a song from her role . And and music video album 's first single Live You 't It 't know it 's not good and a little song , but we 't think that she 's not a lot of the songs . I thought he has not been going to get with the same name and have been a very good album . He also contains the most proud of the " [ ... ] ? I 'm a big @-@ person of the song as 's not to get so I 'm going to be your happy . And I don 't want to do a " She 's not sure the song 's a lot of the song . " In this album , the band called it " the album 's lyrics " , and " It 's not a my own album . " The album 's lyrics are very good for a dance thing that the song is a little more about the album ,
--------------------------------------------------------------------------------



...

💭 Prompt: Tell me a fun fact about space.
💬 Response: s , but she is still known as a single @-@ and @-@ like @-@ style . The most influential and a person of a woman is named for a big thing about a different film of the film The Guardian and the first episode of the album 's lyrics . It was written by the band 's best @-@ disc with a woman on the album 's first week of the same year @-@ season and @-@ ray @-@ winning drama album , while in which he was released on the Billboard 200 with a 5 @-@ year @-@ up for the band 's record of the year . She later described the song as the album 's studio album 's " My Hits " in the band 's second single " A ' " track " and " My Pop " video " and " They " and " This track " and " Don It 't Stop " . The album features the song " Good Up " , which was certified platinum by critics , and directed by Michael Jackson and described the song as a " beautiful
--------------------------------------------------------------------------------



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


💭 Prompt: If you could travel anywhere, where would you go?
💬 Response: ; and that , [ 1 ] ] is a big song to the end of the top ten weeks of the most influential song . They are " Something " is a song that has been shown from the album 's " My It is a song from the album 's music video for the album . It is a song for a song on the album 's single " She 's sure " , but it is " a little @-@ person @-@ off @-@ the @-@ year @-@ hop track " . The album also has a song , and its second single with a " bit of Time " and " I 'm a 's It 's not the most interesting and I 't have been very clear to have a lot of my best and more than the album . " The song is a lot of " What It It 's " and " most interesting @-@ selling " and a sample of The liner notes that it 's seen as " a lot of the best positive than the song " . It
--------------------------------------------------------------------------------



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


💭 Prompt: Explain photosynthesis in simple terms.
💬 Response: i : " I think a bit for his father who are not always more than he thought that the song is . I don 'm my most powerful song to be a big @-@ style film , but I think that you will be able to have been a little more interesting to be a lot of time or if you think it is no one of the album 's best studio album " . " According to Jackson , it is the most likely @-@ one of the album 's best @-@ hop album The Sweet of Music Tour : " The New York Times was not a bit song , so I ’ t going to be a lot of the song , " which was never nominated on a " stunning album 's reviewer " . He appeared on the album 's official version of the song , while the song 's lyrics are going to the movie with the album 's first time . In the band 's track , the song was filmed on a positive review . " On the album 's 's
--------------------------------------------------------------------------------

💭 Prompt: Tell me a short motivational quote.
💬 Resp

# 14 Phase 7a Chat Alignment & Stabilization

In [21]:
# =====================================================
# ✅ Phase-7a — Chat Stabilization Fine-Tune
#  Goal: Fix "domain collapse" and rebalance dialogue flow
# =====================================================

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, TrainerCallback
)
from datasets import load_dataset, concatenate_datasets
from google.colab import drive
import torch, os

# ---- 1️⃣ Mount Drive & Paths
drive.mount("/content/drive", force_remount=True)
base_model_path = "/content/drive/MyDrive/OAIx_3B_state7_ChatSFT/final_chat"
save_dir = "/content/drive/MyDrive/OAIx_3B_state7A_ChatFix"
os.makedirs(save_dir, exist_ok=True)

print(f"✅ Base: {base_model_path}")
print(f"✅ Checkpoints → {save_dir}")

# ---- 2️⃣ Load diverse chat datasets
ultra = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft[:3000]")
alpaca = load_dataset("tatsu-lab/alpaca", split="train[:3000]")
orca_math = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:1000]")

# ---- 3️⃣ Unify into User–Assistant format
def unify(example):
    instr = (
        example.get("instruction") or
        example.get("input") or
        example.get("question") or
        example.get("system_prompt") or
        example.get("prompt") or ""
    )
    resp = (
        example.get("output") or
        example.get("response") or
        example.get("answer") or
        example.get("text") or ""
    )
    if not instr.strip() or not resp.strip():
        return {"text": None}
    return {"text": f"User: {instr.strip()}\nAssistant: {resp.strip()}"}

ultra = ultra.map(unify)
alpaca = alpaca.map(unify)
orca_math = orca_math.map(unify)

ultra = ultra.filter(lambda x: x["text"] is not None)
alpaca = alpaca.filter(lambda x: x["text"] is not None)
orca_math = orca_math.filter(lambda x: x["text"] is not None)

dataset = concatenate_datasets([ultra, alpaca, orca_math]).shuffle(seed=42)
dataset = dataset.filter(lambda x: len(x["text"].split()) < 800)
dataset = dataset.train_test_split(test_size=0.03)

print(f"✅ Chat dataset ready: {len(dataset['train']):,} train | {len(dataset['test']):,} val")

# ---- 4️⃣ Load tokenizer & model
tok = AutoTokenizer.from_pretrained(base_model_path, use_fast=True)
tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
).to("cuda")

print("✅ Model loaded (chat fine-tune ready).")

# ---- 5️⃣ Tokenization + Masking (loss only on assistant)
def tokenize_mask(batch):
    inputs, labels, attn = [], [], []
    for text in batch["text"]:
        if "Assistant:" not in text:
            continue
        parts = text.split("Assistant:")
        user_part = parts[0] + "Assistant:"
        asst_part = parts[1]
        full = user_part + asst_part
        enc = tok(full, truncation=True, max_length=1024, padding="max_length")
        lbl = enc["input_ids"].copy()
        user_len = len(tok(user_part)["input_ids"]) - 1
        lbl[:user_len] = [-100] * user_len
        inputs.append(enc["input_ids"])
        labels.append(lbl)
        attn.append([1]*len(enc["input_ids"]))
    return {"input_ids": inputs, "labels": labels, "attention_mask": attn}

tokenized = dataset.map(tokenize_mask, batched=True, remove_columns=["text"])
print("✅ Tokenization + masking complete.")

# ---- 6️⃣ Data collator
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# ---- 7️⃣ Training config (soft stabilization)
training_args = TrainingArguments(
    output_dir=save_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    max_steps=3500,
    warmup_ratio=0.05,
    weight_decay=0.01,
    bf16=True,
    save_strategy="steps",
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_total_limit=2,
    logging_steps=50,
    report_to="none"
)

# ---- 8️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tok,
)

# ---- 9️⃣ Simple loss logger
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step:<6} | Loss: {logs['loss']:.4f}")
trainer.add_callback(LossPrinterCallback())

# ---- 🔟 Train
print("🚀 Starting Phase-7a (Chat Stabilization)...")
trainer.train()

# ---- 🔚 Save model
final_dir = f"{save_dir}/final_chat7a"
trainer.save_model(final_dir)
tok.save_pretrained(final_dir)
print(f"✅ Final Chat-Fixed model saved to: {final_dir}")


Mounted at /content/drive
✅ Base: /content/drive/MyDrive/OAIx_3B_state7_ChatSFT/final_chat
✅ Checkpoints → /content/drive/MyDrive/OAIx_3B_state7A_ChatFix


Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


✅ Tokenization + masking complete.
🚀 Starting Phase-7a (Chat Stabilization)...


Step,Training Loss,Validation Loss
1000,5.244000,5.453080
2000,5.138100,5.415604
3000,5.180900,5.408329


Step 50     | Loss: 5.6638
Step 100    | Loss: 5.6138
Step 150    | Loss: 5.5204
Step 200    | Loss: 5.4496
Step 250    | Loss: 5.4243
Step 300    | Loss: 5.2941
Step 350    | Loss: 5.3901
Step 400    | Loss: 5.2665
Step 450    | Loss: 5.2874
Step 500    | Loss: 5.2671
Step 550    | Loss: 5.2792
Step 600    | Loss: 5.2176
Step 650    | Loss: 5.1987
Step 700    | Loss: 5.2153
Step 750    | Loss: 5.2043
Step 800    | Loss: 5.1413
Step 850    | Loss: 5.2180
Step 900    | Loss: 5.2608
Step 950    | Loss: 5.2185
Step 1000   | Loss: 5.2440
Step 1050   | Loss: 5.2046
Step 1100   | Loss: 5.2237
Step 1150   | Loss: 5.2293
Step 1200   | Loss: 5.1653
Step 1250   | Loss: 5.1526
Step 1300   | Loss: 5.1899
Step 1350   | Loss: 5.1414
Step 1400   | Loss: 5.1833
Step 1450   | Loss: 5.1465
Step 1500   | Loss: 5.2243
Step 1550   | Loss: 5.1991
Step 1600   | Loss: 5.1613
Step 1650   | Loss: 5.1696
Step 1700   | Loss: 5.1244
Step 1750   | Loss: 5.1160
Step 1800   | Loss: 5.1601
Step 1850   | Loss: 5.1264
S

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ---- 📁 Model yolları (Drive dizinlerini kendine göre kontrol et)
paths = {
    "7a-3000": "/content/drive/MyDrive/OAIx_3B_state7A_ChatFix/checkpoint-3000",
    "7a-3500": "/content/drive/MyDrive/OAIx_3B_state7A_ChatFix/checkpoint-3500",
}

# ---- 🧠 Yükleyici
def load_model(path):
    tok = AutoTokenizer.from_pretrained(path, local_files_only=True)
    model = AutoModelForCausalLM.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        local_files_only=True
    ).to("cuda")
    return tok, model

# ---- 💬 Chat fonksiyonu
def chat(tok, model, prompt, max_new_tokens=180, temperature=0.8, top_p=0.9):
    inputs = tok(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tok.eos_token_id
    )
    return tok.decode(output[0], skip_special_tokens=True)

# ---- 🔹 Prompt listesi
prompts = [
    "User: Hey! How’s your day going so far?\nAssistant:",
    "User: Do you prefer mornings or nights? Why?\nAssistant:",
    "User: Write a short story about a robot who wants to become human.\nAssistant:",
    "User: My friend is sad. What can I do to cheer them up?\nAssistant:",
    "User: Thanks for chatting with me today!\nAssistant:",
]

# ---- 🚀 Test başlat
print("🚀 Starting 4-checkpoint chat comparison...\n")

for model_name, model_path in paths.items():
    print(f"🔹 Loading {model_name}...")
    tok, model = load_model(model_path)

    print(f"✅ {model_name} loaded.\n")
    print("=" * 120)

    for i, p in enumerate(prompts, 1):
        print(f"🧩 Prompt {i}: {p.splitlines()[0]}")
        response = chat(tok, model, p)
        print(f"💬 {model_name} response:\n{response}")
        print("-" * 120)

    print("=" * 120 + "\n")


🚀 Starting 4-checkpoint chat comparison...

🔹 Loading 7a-3000...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 7a-3000 loaded.

🧩 Prompt 1: User: Hey! How’s your day going so far?
💬 7a-3000 response:
User: Hey! How’s your day going so far?
Assistant: to the next day . The film has been made on the third and final film for its most important album release . The following day , it was produced by David H. Martin.com , who was the second season of the album and received positive reviews on the first season of the season . The song was the fifth episode of the song by the show , a " Top rating " , and the song was watched by the album . The band was also in the United States on the US Billboard Hot 100 chart on the Hot 100 and was certified platinum in the United States on March 15 , 2010 . In the US , it was released at number one of the top ten , and the album was certified platinum by an American single by the Billboard Hot 100 . The band was announced that the song was released in the United States . It was the band 's third single on the Hot 100 chart for the week
---------------------------

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 7a-3500 loaded.

🧩 Prompt 1: User: Hey! How’s your day going so far?
💬 7a-3500 response:
User: Hey! How’s your day going so far?
Assistant: as a " big " or " the song " in the " . It was the third single by The Office of the Year 's best @-@ selling album of the UK . It was released on July 17 , 2009 on the Hot 100 , and was released on July 22 , 2010 . The video was nominated for the band 's previous single on the Billboard Hot 100 chart , and the " Love " was released on October 1 , 2006 , in November 2009 , 2006 , and the chart was written by the group 's Top Latin R & BAA . It was also a strong review in the United States on April 15 , 2007 , and its third album sold in the US Hot 100 chart . It is the second single from the United Kingdom on the chart , and was announced at number 15 , and peaked at number 10 on the week 's Hot 100 on April 15 , 2013 . It was released
----------------------------------------------------------------------------------------------------------------